## Final project  of E-commerce course

#### By Orelie Shahar

In [0]:
### I use google colaboratory to run this code - so all the path are empty if you wish to run localy please add the correct path of movielen 100k dataset.

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass

In [8]:
from google.colab import files
uploaded = files.upload()

Saving u1.base to u1.base
Saving u1.test to u1.test
Saving u.data to u.data
Saving u.genre to u.genre
Saving u.info to u.info
Saving u.item to u.item
Saving u.occupation to u.occupation
Saving u.user to u.user


### Part 1 - MF and GMF

#### Matrix Factorization

In [0]:
import numpy as np
import pandas as pd
import os
import tensorflow as T
import keras
from keras import backend as K
from keras import initializers
from keras.initializers import RandomNormal
from keras.models import Sequential, Model, load_model, save_model
from keras.layers.core import Dense , Dropout, Lambda, Activation
from keras.layers import Embedding, Input, Dense, merge, Reshape, Flatten
from keras.optimizers import Adagrad, Adam, SGD, RMSprop
from keras.regularizers import l2
from keras.layers import Multiply, Concatenate
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from time import time
import multiprocessing as mp
import sys
import math
import argparse
import matplotlib.pyplot as plt

In [10]:
##add your path here if you run localy and not with google colaboratry
MOVIELENS_DIR = ''
RATING_DATA_FILE = 'u1.base'
RATINGS_CSV_FILE = 'u_emb.data'
MODEL_WEIGHTS_FILE = 'u_emb_weights.h5'


ratings = pd.read_csv(os.path.join(MOVIELENS_DIR, RATING_DATA_FILE), 
                    sep='\t', 
                    engine='python', 
                    encoding='latin-1',
                    names=['userid', 'movieid', 'rating', 'timestamp'])
max_userid = ratings['userid'].drop_duplicates().max()
max_movieid = ratings['movieid'].drop_duplicates().max()
ratings['user_emb_id'] = ratings['userid'] - 1
ratings['movie_emb_id'] = ratings['movieid'] - 1
print(str(len(ratings))+' ratings loaded - for train')
ratings.to_csv(RATINGS_CSV_FILE, 
               sep='\t', 
               header=True, 
               encoding='latin-1', 
               columns=['userid', 'movieid', 'rating', 'timestamp', 'user_emb_id', 'movie_emb_id'])


ratings = pd.read_csv(RATINGS_CSV_FILE, 
                      sep='\t', 
                      encoding='latin-1', 
                      usecols=['userid', 'movieid', 'user_emb_id', 'movie_emb_id', 'rating'])
max_userid = ratings['userid'].drop_duplicates().max()
max_movieid = ratings['movieid'].drop_duplicates().max()

shuffled_ratings = ratings.sample(frac=1.)
Users = shuffled_ratings['user_emb_id'].values
Movies = shuffled_ratings['movie_emb_id'].values
Ratings = shuffled_ratings['rating'].values


80000 ratings loaded - for train


In [11]:
MOVIELENS_DIR = ''
RATING_DATA_FILE = 'u1.test'
RATINGS_CSV_FILE = 'u_emb1.data'
MODEL_WEIGHTS_FILE = 'u_emb1_weights.h5'


ratings1 = pd.read_csv(os.path.join(MOVIELENS_DIR, RATING_DATA_FILE), 
                    sep='\t', 
                    engine='python', 
                    encoding='latin-1',
                    names=['userid', 'movieid', 'rating', 'timestamp'])
max_userid1 = ratings1['userid'].drop_duplicates().max()
max_movieid1 = ratings1['movieid'].drop_duplicates().max()
ratings1['user_emb_id'] = ratings1['userid'] - 1
ratings1['movie_emb_id'] = ratings1['movieid'] - 1
print(str(len(ratings1))+' ratings loaded for tests')
ratings1.to_csv(RATINGS_CSV_FILE, 
               sep='\t', 
               header=True, 
               encoding='latin-1', 
               columns=['userid', 'movieid', 'rating', 'timestamp', 'user_emb_id', 'movie_emb_id'])



ratings1 = pd.read_csv(RATINGS_CSV_FILE, 
                      sep='\t', 
                      encoding='latin-1', 
                      usecols=['userid', 'movieid', 'user_emb_id', 'movie_emb_id', 'rating'])
max_userid1 = ratings1['userid'].drop_duplicates().max()
max_movieid1 = ratings1['movieid'].drop_duplicates().max()


shuffled_ratings1 = ratings1.sample(frac=1.)
Users1 = shuffled_ratings1['user_emb_id'].values
Movies1 = shuffled_ratings1['movie_emb_id'].values
Ratings1 = shuffled_ratings1['rating'].values


20000 ratings loaded for tests


### we will calculate the result and MAE with:
#### embedding of: user-item, item-user
#### drop-out of: 0.1, 0.5

In [0]:
#embedding item-user
def get_model_item_user(num_users, num_items, latent_dim,dropout_size):
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')

    MF_Embedding_User = Embedding(input_dim = num_users, output_dim = latent_dim, name = 'user_embedding', input_length=1)
    MF_Embedding_Item = Embedding(input_dim = num_items, output_dim = latent_dim, name = 'item_embedding', input_length=1)   
    
    # Crucial to flatten an embedding vector!
    user_latent = Flatten()(MF_Embedding_User(user_input))
    item_latent = Flatten()(MF_Embedding_Item(item_input))
    
    # Element-wise product of user and item embeddings
    prediction = merge([item_latent,user_latent], mode = 'dot')
    prediction = Dropout(dropout_size)(prediction)

    
    model = Model(input=[item_input, user_input], output=prediction)

    return model
  
  
  
  
#embedding user-item
def get_model_user_item(num_users, num_items, latent_dim,dropout_size):
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')

    MF_Embedding_User = Embedding(input_dim = num_users, output_dim = latent_dim, name = 'user_embedding', input_length=1)
    MF_Embedding_Item = Embedding(input_dim = num_items, output_dim = latent_dim, name = 'item_embedding', input_length=1)   
    
    # Crucial to flatten an embedding vector!
    user_latent = Flatten()(MF_Embedding_User(user_input))
    item_latent = Flatten()(MF_Embedding_Item(item_input))
    
    # Element-wise product of user and item embeddings
    prediction = merge([user_latent,item_latent], mode = 'dot')
    prediction = Dropout(dropout_size)(prediction)

    
    model = Model(input=[ user_input,item_input], output=prediction)

    return model
  


### embedding = Item-User , Dropout = 0.5


In [13]:


K_LATENT = 20
dropout = 0.5

MF_model = get_model_item_user(max_userid,max_movieid,K_LATENT,dropout)
MF_model.compile(loss='mae', optimizer='adamax',metrics=['mae'])
import time
ini = time.time()
callbacks = [EarlyStopping('val_loss', patience=3), 
             ModelCheckpoint(MODEL_WEIGHTS_FILE, save_best_only=True)]
history = MF_model.fit([Movies, Users], Ratings, nb_epoch=100,  verbose=1, validation_data=([Movies1,Users1], Ratings1), callbacks=callbacks)


end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
/usr/local/lib/python3.6/dist-packages/keras/legacy/layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("dr...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if sys.path[0] == '':


Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 4s 54us/step - loss: 3.5278 - mean_absolute_error: 3.5278 - val_loss: 3.5338 - val_mean_absolute_error: 3.5338
Epoch 2/100
80000/80000 [==============================] - 4s 51us/step - loss: 3.5004 - mean_absolute_error: 3.5004 - val_loss: 3.4788 - val_mean_absolute_error: 3.4788
Epoch 3/100
80000/80000 [==============================] - 4s 51us/step - loss: 3.3314 - mean_absolute_error: 3.3314 - val_loss: 3.2706 - val_mean_absolute_error: 3.2706
Epoch 4/100
41024/80000 [==============>...............] - ETA: 1s - loss: 3.0865 - mean_absolute_error: 3.086580000/80000 [==============================] - 4s 51us/step - loss: 2.9961 - mean_absolute_error: 2.9961 - val_loss: 2.9389 - val_mean_absolute_error: 2.9389
Epoch 5/100
80000/80000 [==============================] - 4s 50us/step - loss: 2.7156 - mean_absolute_error: 2.7156 - val_loss: 2.6372 - val_mean_absolute_error: 2.6372
E

74400/80000 [==========================>...] - ETA: 0s - loss: 2.1174 - mean_absolute_error: 2.117480000/80000 [==============================] - 4s 51us/step - loss: 2.1180 - mean_absolute_error: 2.1180 - val_loss: 1.8067 - val_mean_absolute_error: 1.8067
Epoch 46/100
80000/80000 [==============================] - 4s 51us/step - loss: 2.1076 - mean_absolute_error: 2.1076 - val_loss: 1.8017 - val_mean_absolute_error: 1.8017
Epoch 47/100
80000/80000 [==============================] - 4s 52us/step - loss: 2.1126 - mean_absolute_error: 2.1126 - val_loss: 1.8030 - val_mean_absolute_error: 1.8030
Epoch 48/100
75648/80000 [===========================>..] - ETA: 0s - loss: 2.1125 - mean_absolute_error: 2.112580000/80000 [==============================] - 4s 53us/step - loss: 2.1111 - mean_absolute_error: 2.1111 - val_loss: 1.8019 - val_mean_absolute_error: 1.8019
Epoch 49/100
80000/80000 [==============================] - 4s 53us/step - loss: 2.1157 - mean_absolute_error: 2.1157 - val_loss: 1

In [25]:
#score = model.evaluate(x_test, y_test, verbose=0)
score =MF_model.evaluate([Movies1,Users1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])


20000/20000 [==============================] - 0s 23us/step
Test MSE: 1.8032565315246583
Test MAE: 1.8032565315246583


### embedding = Item-User , Dropout = 0.1


In [26]:
import time

K_LATENT = 20
dropout = 0.1

MF_model = get_model_item_user(max_userid,max_movieid,K_LATENT,dropout)
MF_model.compile(loss='mae', optimizer='adamax',metrics=['mae'])

ini = time.time()
callbacks = [EarlyStopping('val_loss', patience=3), 
             ModelCheckpoint(MODEL_WEIGHTS_FILE, save_best_only=True)]
history = MF_model.fit([Movies, Users], Ratings, nb_epoch=100, validation_split=.1, verbose=1,validation_data=([Movies1,Users1], Ratings1), callbacks=callbacks, batch_size = 32)


end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  
/usr/local/lib/python3.6/dist-packages/keras/legacy/layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("dr...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if sys.path[0] == '':


Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 4s 53us/step - loss: 3.5197 - mean_absolute_error: 3.5197 - val_loss: 3.5010 - val_mean_absolute_error: 3.5010
Epoch 2/100
80000/80000 [==============================] - 4s 51us/step - loss: 3.2684 - mean_absolute_error: 3.2684 - val_loss: 3.1022 - val_mean_absolute_error: 3.1022
Epoch 3/100
80000/80000 [==============================] - 4s 51us/step - loss: 2.5328 - mean_absolute_error: 2.5328 - val_loss: 2.3864 - val_mean_absolute_error: 2.3864
Epoch 4/100
40640/80000 [==============>...............] - ETA: 1s - loss: 2.0292 - mean_absolute_error: 2.029280000/80000 [==============================] - 4s 51us/step - loss: 1.9278 - mean_absolute_error: 1.9278 - val_loss: 1.8471 - val_mean_absolute_error: 1.8471
Epoch 5/100
80000/80000 [==============================] - 4s 51us/step - loss: 1.5971 - mean_absolute_error: 1.5971 - val_loss: 1.5125 - val_mean_absolute_error: 1.5125
E

70752/80000 [=========================>....] - ETA: 0s - loss: 0.9544 - mean_absolute_error: 0.954480000/80000 [==============================] - 4s 51us/step - loss: 0.9546 - mean_absolute_error: 0.9546 - val_loss: 0.8084 - val_mean_absolute_error: 0.8084
Epoch 46/100
80000/80000 [==============================] - 4s 51us/step - loss: 0.9481 - mean_absolute_error: 0.9481 - val_loss: 0.8081 - val_mean_absolute_error: 0.8081
Epoch 47/100
80000/80000 [==============================] - 4s 51us/step - loss: 0.9472 - mean_absolute_error: 0.9472 - val_loss: 0.8072 - val_mean_absolute_error: 0.8072
Epoch 48/100
78080/80000 [============================>.] - ETA: 0s - loss: 0.9432 - mean_absolute_error: 0.943280000/80000 [==============================] - 4s 51us/step - loss: 0.9425 - mean_absolute_error: 0.9425 - val_loss: 0.8034 - val_mean_absolute_error: 0.8034
Epoch 49/100
80000/80000 [==============================] - 4s 51us/step - loss: 0.9446 - mean_absolute_error: 0.9446 - val_loss: 0

In [27]:
score =MF_model.evaluate([Movies1,Users1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

20000/20000 [==============================] - 0s 23us/step
Test MSE: 0.8021336905479431
Test MAE: 0.8021336905479431


### embedding= User-item , Dropout = 0.5


In [28]:

import time

K_LATENT = 20
dropout = 0.5

MF_model = get_model_user_item(max_userid,max_movieid,K_LATENT,dropout)
MF_model.compile(loss='mae', optimizer='adamax',metrics=['mae'])

ini = time.time()
callbacks = [EarlyStopping('val_loss', patience=3), 
             ModelCheckpoint(MODEL_WEIGHTS_FILE, save_best_only=True)]
history = MF_model.fit([Users,Movies], Ratings, nb_epoch=100, validation_split=.1, verbose=1,validation_data=([Users1,Movies1], Ratings1), callbacks=callbacks, batch_size = 32)


end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python3.6/dist-packages/keras/legacy/layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("dr...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  del sys.path[0]


Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 4s 53us/step - loss: 3.5272 - mean_absolute_error: 3.5272 - val_loss: 3.5321 - val_mean_absolute_error: 3.5321
Epoch 2/100
80000/80000 [==============================] - 4s 51us/step - loss: 3.4902 - mean_absolute_error: 3.4902 - val_loss: 3.4642 - val_mean_absolute_error: 3.4642
Epoch 3/100
80000/80000 [==============================] - 4s 51us/step - loss: 3.3040 - mean_absolute_error: 3.3040 - val_loss: 3.2420 - val_mean_absolute_error: 3.2420
Epoch 4/100
41024/80000 [==============>...............] - ETA: 1s - loss: 3.0450 - mean_absolute_error: 3.045080000/80000 [==============================] - 4s 51us/step - loss: 2.9641 - mean_absolute_error: 2.9641 - val_loss: 2.9103 - val_mean_absolute_error: 2.9103
Epoch 5/100
80000/80000 [==============================] - 4s 51us/step - loss: 2.6871 - mean_absolute_error: 2.6871 - val_loss: 2.6097 - val_mean_absolute_error: 2.6097
E

77632/80000 [============================>.] - ETA: 0s - loss: 2.1110 - mean_absolute_error: 2.111080000/80000 [==============================] - 4s 51us/step - loss: 2.1110 - mean_absolute_error: 2.1110 - val_loss: 1.8035 - val_mean_absolute_error: 1.8035
Epoch 45/100
80000/80000 [==============================] - 4s 51us/step - loss: 2.1049 - mean_absolute_error: 2.1049 - val_loss: 1.8054 - val_mean_absolute_error: 1.8054
Epoch 46/100
80000/80000 [==============================] - 4s 52us/step - loss: 2.1145 - mean_absolute_error: 2.1145 - val_loss: 1.8017 - val_mean_absolute_error: 1.8017
Epoch 47/100
80000/80000 [==============================] - 4s 51us/step - loss: 2.1147 - mean_absolute_error: 2.1147 - val_loss: 1.7994 - val_mean_absolute_error: 1.7994
Epoch 48/100
80000/80000 [==============================] - 4s 51us/step - loss: 2.1165 - mean_absolute_error: 2.1165 - val_loss: 1.8078 - val_mean_absolute_error: 1.8078
Epoch 49/100
80000/80000 [==============================] -

In [31]:
score =MF_model.evaluate([Users1,Movies1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

20000/20000 [==============================] - 0s 23us/step
Test MSE: 1.8085132280349732
Test MAE: 1.8085132280349732


#### embedding = User-item , Dropout = 0.1


In [32]:

import time

K_LATENT = 20
dropout = 0.1

MF_model = get_model_user_item(max_userid,max_movieid,K_LATENT,dropout)
MF_model.compile(loss='mae', optimizer='adamax',metrics=['mae'])

ini = time.time()
callbacks = [EarlyStopping('val_loss', patience=3), 
             ModelCheckpoint(MODEL_WEIGHTS_FILE, save_best_only=True)]
history = MF_model.fit([Users,Movies], Ratings, nb_epoch=100, validation_split=.1, verbose=1,validation_data=([Users1,Movies1], Ratings1), callbacks=callbacks, batch_size = 32)


end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python3.6/dist-packages/keras/legacy/layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("dr...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  del sys.path[0]


Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 4s 54us/step - loss: 3.5207 - mean_absolute_error: 3.5207 - val_loss: 3.5050 - val_mean_absolute_error: 3.5050
Epoch 2/100
80000/80000 [==============================] - 4s 51us/step - loss: 3.3029 - mean_absolute_error: 3.3029 - val_loss: 3.1503 - val_mean_absolute_error: 3.1503
Epoch 3/100
80000/80000 [==============================] - 4s 51us/step - loss: 2.6035 - mean_absolute_error: 2.6035 - val_loss: 2.4442 - val_mean_absolute_error: 2.4442
Epoch 4/100
40256/80000 [==============>...............] - ETA: 1s - loss: 2.0799 - mean_absolute_error: 2.079980000/80000 [==============================] - 4s 51us/step - loss: 1.9647 - mean_absolute_error: 1.9647 - val_loss: 1.8859 - val_mean_absolute_error: 1.8859
Epoch 5/100
80000/80000 [==============================] - 4s 51us/step - loss: 1.6165 - mean_absolute_error: 1.6165 - val_loss: 1.5401 - val_mean_absolute_error: 1.5401
E

79552/80000 [============================>.] - ETA: 0s - loss: 0.9521 - mean_absolute_error: 0.952180000/80000 [==============================] - 4s 52us/step - loss: 0.9520 - mean_absolute_error: 0.9520 - val_loss: 0.8084 - val_mean_absolute_error: 0.8084
Epoch 45/100
80000/80000 [==============================] - 4s 52us/step - loss: 0.9473 - mean_absolute_error: 0.9473 - val_loss: 0.8068 - val_mean_absolute_error: 0.8068
Epoch 46/100
80000/80000 [==============================] - 4s 52us/step - loss: 0.9507 - mean_absolute_error: 0.9507 - val_loss: 0.8058 - val_mean_absolute_error: 0.8058
Epoch 47/100
78496/80000 [============================>.] - ETA: 0s - loss: 0.9401 - mean_absolute_error: 0.940180000/80000 [==============================] - 4s 52us/step - loss: 0.9394 - mean_absolute_error: 0.9394 - val_loss: 0.8042 - val_mean_absolute_error: 0.8042
Epoch 48/100
80000/80000 [==============================] - 4s 51us/step - loss: 0.9449 - mean_absolute_error: 0.9449 - val_loss: 0

In [33]:
score =MF_model.evaluate([Users1,Movies1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

20000/20000 [==============================] - 0s 24us/step
Test MSE: 0.8035083614349365
Test MAE: 0.8035083614349365


### End of MF part - all result in the pdf.

#### General Matrix Factorization - GMF

In [34]:
##add your path here if you run localy and not with google colaboratry
MOVIELENS_DIR = ''
RATING_DATA_FILE = 'u1.base'
RATINGS_CSV_FILE = 'u_emb.data'
MODEL_WEIGHTS_FILE = 'u_emb_weights.h5'


ratings = pd.read_csv(os.path.join(MOVIELENS_DIR, RATING_DATA_FILE), 
                    sep='\t', 
                    engine='python', 
                    encoding='latin-1',
                    names=['userid', 'movieid', 'rating', 'timestamp'])
max_userid = ratings['userid'].drop_duplicates().max()
max_movieid = ratings['movieid'].drop_duplicates().max()
ratings['user_emb_id'] = ratings['userid'] - 1
ratings['movie_emb_id'] = ratings['movieid'] - 1
print(str(len(ratings))+' ratings loaded - for train')
ratings.to_csv(RATINGS_CSV_FILE, 
               sep='\t', 
               header=True, 
               encoding='latin-1', 
               columns=['userid', 'movieid', 'rating', 'timestamp', 'user_emb_id', 'movie_emb_id'])


ratings = pd.read_csv(RATINGS_CSV_FILE, 
                      sep='\t', 
                      encoding='latin-1', 
                      usecols=['userid', 'movieid', 'user_emb_id', 'movie_emb_id', 'rating'])
max_userid = ratings['userid'].drop_duplicates().max()
max_movieid = ratings['movieid'].drop_duplicates().max()

shuffled_ratings = ratings.sample(frac=1.)
Users = shuffled_ratings['user_emb_id'].values
Movies = shuffled_ratings['movie_emb_id'].values
Ratings = shuffled_ratings['rating'].values

80000 ratings loaded - for train


In [35]:
MOVIELENS_DIR = ''
RATING_DATA_FILE = 'u1.test'
RATINGS_CSV_FILE = 'u_emb1.data'
MODEL_WEIGHTS_FILE = 'u_emb1_weights.h5'
MODEL_WEIGHTS_FILE_CORE = 'u_emb_weights'

ratings1 = pd.read_csv(os.path.join(MOVIELENS_DIR, RATING_DATA_FILE), 
                    sep='\t', 
                    engine='python', 
                    encoding='latin-1',
                    names=['userid', 'movieid', 'rating', 'timestamp'])
max_userid1 = ratings1['userid'].drop_duplicates().max()
max_movieid1 = ratings1['movieid'].drop_duplicates().max()
ratings1['user_emb_id'] = ratings1['userid'] - 1
ratings1['movie_emb_id'] = ratings1['movieid'] - 1
print(str(len(ratings1))+' ratings loaded for tests')
ratings1.to_csv(RATINGS_CSV_FILE, 
               sep='\t', 
               header=True, 
               encoding='latin-1', 
               columns=['userid', 'movieid', 'rating', 'timestamp', 'user_emb_id', 'movie_emb_id'])



ratings1 = pd.read_csv(RATINGS_CSV_FILE, 
                      sep='\t', 
                      encoding='latin-1', 
                      usecols=['userid', 'movieid', 'user_emb_id', 'movie_emb_id', 'rating'])
max_userid1 = ratings1['userid'].drop_duplicates().max()
max_movieid1 = ratings1['movieid'].drop_duplicates().max()


shuffled_ratings1 = ratings1.sample(frac=1.)
Users1 = shuffled_ratings1['user_emb_id'].values
Movies1 = shuffled_ratings1['movie_emb_id'].values
Ratings1 = shuffled_ratings1['rating'].values



20000 ratings loaded for tests


In [0]:
#embedding user-item
def get_gmf_model(num_users, num_items, latent_dim,do):
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')

    MF_Embedding_User = Embedding(input_dim = num_users, output_dim = latent_dim, name = 'user_embedding', input_length=1)
    MF_Embedding_Item = Embedding(input_dim = num_items, output_dim = latent_dim, name = 'item_embedding', input_length=1)   
    
    # Crucial to flatten an embedding vector!
    user_latent = Flatten()(MF_Embedding_User(user_input))
    item_latent = Flatten()(MF_Embedding_Item(item_input))
    
    # Element-wise product of user and item embeddings
    hidden1 = Multiply()([user_latent, item_latent])
    drop = Dropout(do)(hidden1)
    prediction = Dense(1, activation='relu', kernel_initializer='lecun_uniform', name = 'prediction')(drop)
    
    
    model = Model(input=[user_input, item_input], output=prediction)

    return model

### we will calculate the result and MAE with:
#### embedding of: user-item
#### drop-out of: 0.1, 0.5

### embedding  = User-Item , Dropout  = 0.5


In [37]:
K_LATENT = 20
hidden_dim = 20
do = 0.5
GMF_model = get_gmf_model(max_userid,max_movieid,K_LATENT,do)
GMF_model.compile(loss='mse',optimizer='adamax',metrics=['mae'])


ini = time.time()
callbacks_gmf = [EarlyStopping('val_loss', patience=3), ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_gmf_'+str(do)+'_'+str(K_LATENT)+'.h5', save_best_only=True)]
history = GMF_model.fit([Users, Movies], Ratings, nb_epoch=100, validation_split=.1,validation_data=([Users1,Movies1], Ratings1), verbose=1, callbacks=callbacks_gmf, batch_size = 32)

end = time.time()
print ("Total time for embedding = User-Item , Dropout = 0.5: ",end-ini)
print("MAE for embedding = User-Item , Dropout = 0.5: " , min(history.history['val_loss']))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("pr...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # Remove the CWD from sys.path while we load stuff.


Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 5s 58us/step - loss: 4.7144 - mean_absolute_error: 1.8064 - val_loss: 1.3486 - val_mean_absolute_error: 0.9836
Epoch 2/100
80000/80000 [==============================] - 4s 53us/step - loss: 1.2510 - mean_absolute_error: 0.9401 - val_loss: 1.3291 - val_mean_absolute_error: 0.9684
Epoch 3/100
80000/80000 [==============================] - 4s 52us/step - loss: 1.2409 - mean_absolute_error: 0.9345 - val_loss: 1.3172 - val_mean_absolute_error: 0.9642
Epoch 4/100
36704/80000 [============>.................] - ETA: 2s - loss: 1.2227 - mean_absolute_error: 0.926680000/80000 [==============================] - 4s 52us/step - loss: 1.2086 - mean_absolute_error: 0.9198 - val_loss: 1.2798 - val_mean_absolute_error: 0.9471
Epoch 5/100
80000/80000 [==============================] - 4s 52us/step - loss: 1.1476 - mean_absolute_error: 0.8886 - val_loss: 1.2199 - val_mean_absolute_error: 0.9174
E

77952/80000 [============================>.] - ETA: 0s - loss: 0.7393 - mean_absolute_error: 0.672380000/80000 [==============================] - 4s 53us/step - loss: 0.7388 - mean_absolute_error: 0.6720 - val_loss: 0.9112 - val_mean_absolute_error: 0.7539
Epoch 44/100
80000/80000 [==============================] - 4s 53us/step - loss: 0.7328 - mean_absolute_error: 0.6704 - val_loss: 0.9105 - val_mean_absolute_error: 0.7537
Epoch 45/100
80000/80000 [==============================] - 4s 52us/step - loss: 0.7362 - mean_absolute_error: 0.6714 - val_loss: 0.9099 - val_mean_absolute_error: 0.7533
Epoch 46/100
78080/80000 [============================>.] - ETA: 0s - loss: 0.7310 - mean_absolute_error: 0.668580000/80000 [==============================] - 4s 52us/step - loss: 0.7303 - mean_absolute_error: 0.6682 - val_loss: 0.9092 - val_mean_absolute_error: 0.7532
Epoch 47/100
80000/80000 [==============================] - 4s 53us/step - loss: 0.7300 - mean_absolute_error: 0.6675 - val_loss: 0

In [38]:
score =GMF_model.evaluate([Users1,Movies1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

20000/20000 [==============================] - 0s 24us/step
Test MSE: 0.9016449202060699
Test MAE: 0.7488374603748321


### embedding = User- Item , Dropout = 0.1

In [39]:
K_LATENT = 20
hidden_dim = 20
do = 0.1
GMF_model = get_gmf_model(max_userid,max_movieid,K_LATENT,do)
GMF_model.compile(loss='mse',optimizer='adamax',metrics=['mae'])


ini = time.time()
callbacks_gmf = [EarlyStopping('val_loss', patience=3), ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_gmf_'+str(do)+'_'+str(K_LATENT)+'.h5', save_best_only=True)]
history = GMF_model.fit([Users, Movies], Ratings, nb_epoch=100, validation_split=.1,validation_data=([Users1,Movies1], Ratings1), verbose=1, callbacks=callbacks_gmf, batch_size = 32)

end = time.time()
print ("Total time for embedding = User-Item , Dropout = 0.5: ",end-ini)
print("MAE for embedding = User-Item , Dropout = 0.5: " , min(history.history['val_loss']))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("pr...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # Remove the CWD from sys.path while we load stuff.


Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 5s 57us/step - loss: 4.6975 - mean_absolute_error: 1.8012 - val_loss: 1.3466 - val_mean_absolute_error: 0.9829
Epoch 2/100
80000/80000 [==============================] - 4s 53us/step - loss: 1.2252 - mean_absolute_error: 0.9290 - val_loss: 1.2812 - val_mean_absolute_error: 0.9470
Epoch 3/100
80000/80000 [==============================] - 4s 53us/step - loss: 1.1263 - mean_absolute_error: 0.8775 - val_loss: 1.1799 - val_mean_absolute_error: 0.8959
Epoch 4/100
35168/80000 [============>.................] - ETA: 2s - loss: 1.0641 - mean_absolute_error: 0.840980000/80000 [==============================] - 4s 53us/step - loss: 1.0389 - mean_absolute_error: 0.8271 - val_loss: 1.1171 - val_mean_absolute_error: 0.8582
Epoch 5/100
80000/80000 [==============================] - 4s 53us/step - loss: 0.9802 - mean_absolute_error: 0.7928 - val_loss: 1.0726 - val_mean_absolute_error: 0.8308
E

In [41]:
score =GMF_model.evaluate([Users1,Movies1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

20000/20000 [==============================] - 0s 24us/step
Test MSE: 0.9381665133953094
Test MAE: 0.7613480550765991


### End of GMF part - all result in the pdf.

#### result for the non-Personalized based on rating avrege from HW2 in the pdf. 

### Part 2 - neural collaborative filtering - NCF

In [42]:
##add your path here if you run localy and not with google colaboratry
MOVIELENS_DIR = ''
RATING_DATA_FILE = 'u1.base'
RATINGS_CSV_FILE = 'u_emb.data'
MODEL_WEIGHTS_FILE = 'u_emb_weights.h5'


ratings = pd.read_csv(os.path.join(MOVIELENS_DIR, RATING_DATA_FILE), 
                    sep='\t', 
                    engine='python', 
                    encoding='latin-1',
                    names=['userid', 'movieid', 'rating', 'timestamp'])
max_userid = ratings['userid'].drop_duplicates().max()
max_movieid = ratings['movieid'].drop_duplicates().max()
ratings['user_emb_id'] = ratings['userid'] - 1
ratings['movie_emb_id'] = ratings['movieid'] - 1
print(str(len(ratings))+' ratings loaded - for train')
ratings.to_csv(RATINGS_CSV_FILE, 
               sep='\t', 
               header=True, 
               encoding='latin-1', 
               columns=['userid', 'movieid', 'rating', 'timestamp', 'user_emb_id', 'movie_emb_id'])


ratings = pd.read_csv(RATINGS_CSV_FILE, 
                      sep='\t', 
                      encoding='latin-1', 
                      usecols=['userid', 'movieid', 'user_emb_id', 'movie_emb_id', 'rating'])
max_userid = ratings['userid'].drop_duplicates().max()
max_movieid = ratings['movieid'].drop_duplicates().max()

shuffled_ratings = ratings.sample(frac=1.)
Users = shuffled_ratings['user_emb_id'].values
Movies = shuffled_ratings['movie_emb_id'].values
Ratings = shuffled_ratings['rating'].values

80000 ratings loaded - for train


In [43]:
MOVIELENS_DIR = ''
RATING_DATA_FILE = 'u1.test'
RATINGS_CSV_FILE = 'u_emb1.data'
MODEL_WEIGHTS_FILE = 'u_emb1_weights.h5'
MODEL_WEIGHTS_FILE_CORE = 'u_emb_weights'

ratings1 = pd.read_csv(os.path.join(MOVIELENS_DIR, RATING_DATA_FILE), 
                    sep='\t', 
                    engine='python', 
                    encoding='latin-1',
                    names=['userid', 'movieid', 'rating', 'timestamp'])
max_userid1 = ratings1['userid'].drop_duplicates().max()
max_movieid1 = ratings1['movieid'].drop_duplicates().max()
ratings1['user_emb_id'] = ratings1['userid'] - 1
ratings1['movie_emb_id'] = ratings1['movieid'] - 1
print(str(len(ratings1))+' ratings loaded for tests')
ratings1.to_csv(RATINGS_CSV_FILE, 
               sep='\t', 
               header=True, 
               encoding='latin-1', 
               columns=['userid', 'movieid', 'rating', 'timestamp', 'user_emb_id', 'movie_emb_id'])



ratings1 = pd.read_csv(RATINGS_CSV_FILE, 
                      sep='\t', 
                      encoding='latin-1', 
                      usecols=['userid', 'movieid', 'user_emb_id', 'movie_emb_id', 'rating'])
max_userid1 = ratings1['userid'].drop_duplicates().max()
max_movieid1 = ratings1['movieid'].drop_duplicates().max()


shuffled_ratings1 = ratings1.sample(frac=1.)
Users1 = shuffled_ratings1['user_emb_id'].values
Movies1 = shuffled_ratings1['movie_emb_id'].values
Ratings1 = shuffled_ratings1['rating'].values



20000 ratings loaded for tests


In [0]:
# hidden_dim = layer size, do  = drop-out
def get_ncf_model(num_users, num_items, latent_dim,hidden_dim,do,activation_name):
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')

    MF_Embedding_User = Embedding(input_dim = num_users, output_dim = latent_dim, name = 'user_embedding', input_length=1)
    MF_Embedding_Item = Embedding(input_dim = num_items, output_dim = latent_dim, name = 'item_embedding', input_length=1)   
    
    # Crucial to flatten an embedding vector!
    user_latent = Flatten()(MF_Embedding_User(user_input))
    item_latent = Flatten()(MF_Embedding_Item(item_input))
    
    # Element-wise product of user and item embeddings
    conc = Concatenate()([user_latent, item_latent])
    drop = Dropout(0.5)(conc)
    hid1 = Dense(hidden_dim, activation=activation_name)(conc)
    drop2  = Dropout(do)(hid1)
    prediction = Dense(1, activation=activation_name, kernel_initializer='lecun_uniform', name = 'prediction')(drop2)
    prediction = Dropout(do)(prediction)

    
    model = Model(input=[user_input, item_input], output=prediction)

    return model

### we will calculate the result and MAE with:
#### Size of Layer: 8, 16, 50, 100
#### Optimizer = sgd, adam, Nadam, Adadelta
#### Function Loss = mean absolute error, mean squared error,  mean_absolute_percentage_error
#### Activation Function = relu, tanh, linear, selu
#### drop-out of: 0.1, 0.5

### Size of Layer: 8 

In [45]:
K_LATENT = 20
hidden_dim = 8
do = 0.5
oprimaizer_name = 'adamax'
loss_function = 'mse'
activ_func ='relu'

NCF_model = get_ncf_model(max_userid,max_movieid,K_LATENT,hidden_dim,do,activ_func)
NCF_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()
callbacks_ncf = [EarlyStopping('val_loss', patience=5), ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncf_'+str(do)+'_'+str(K_LATENT)+'_'+str(hidden_dim)+'.h5', save_best_only=True)]
history = NCF_model.fit([Users, Movies], Ratings, nb_epoch=100,validation_data=([Users1,Movies1], Ratings1), validation_split=.1, verbose=1, callbacks=callbacks_ncf, batch_size = 32)

end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("dr...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if sys.path[0] == '':


Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 5s 62us/step - loss: 8.0544 - mean_absolute_error: 2.3775 - val_loss: 4.5824 - val_mean_absolute_error: 1.9155
Epoch 2/100
80000/80000 [==============================] - 5s 57us/step - loss: 7.4111 - mean_absolute_error: 2.1865 - val_loss: 4.2931 - val_mean_absolute_error: 1.8560
Epoch 3/100
80000/80000 [==============================] - 5s 57us/step - loss: 7.3733 - mean_absolute_error: 2.1688 - val_loss: 4.2839 - val_mean_absolute_error: 1.8548
Epoch 4/100
27104/80000 [=========>....................] - ETA: 2s - loss: 7.3212 - mean_absolute_error: 2.158080000/80000 [==============================] - 5s 57us/step - loss: 7.3567 - mean_absolute_error: 2.1641 - val_loss: 4.1772 - val_mean_absolute_error: 1.8280
Epoch 5/100
80000/80000 [==============================] - 5s 58us/step - loss: 7.3369 - mean_absolute_error: 2.1586 - val_loss: 4.1448 - val_mean_absolute_error: 1.8201
E

In [46]:
score =NCF_model.evaluate([Users1,Movies1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

20000/20000 [==============================] - 1s 29us/step
Test MSE: 4.216665811920166
Test MAE: 1.8366843547821046


### Size of Layer: 16 

In [47]:
K_LATENT = 20
hidden_dim = 16
do = 0.5
ac = 'relu'
oprimaizer_name = 'adamax'
loss_function = 'mse'
activ_func ='relu'

NCF_model = get_ncf_model(max_userid,max_movieid,K_LATENT,hidden_dim,do,activ_func)
NCF_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()
callbacks_ncf = [EarlyStopping('val_loss', patience=5), ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncf_'+str(do)+'_'+str(K_LATENT)+'_'+str(hidden_dim)+'.h5', save_best_only=True)]
history = NCF_model.fit([Users, Movies], Ratings, nb_epoch=100,validation_data=([Users1,Movies1], Ratings1), validation_split=.1, verbose=1, callbacks=callbacks_ncf, batch_size = 32)

end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("dr...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  del sys.path[0]


Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 5s 63us/step - loss: 8.0391 - mean_absolute_error: 2.3706 - val_loss: 4.5439 - val_mean_absolute_error: 1.9064
Epoch 2/100
80000/80000 [==============================] - 5s 57us/step - loss: 7.3491 - mean_absolute_error: 2.1764 - val_loss: 4.2720 - val_mean_absolute_error: 1.8475
Epoch 3/100
80000/80000 [==============================] - 5s 57us/step - loss: 7.3298 - mean_absolute_error: 2.1554 - val_loss: 4.1804 - val_mean_absolute_error: 1.8267
Epoch 4/100
24960/80000 [========>.....................] - ETA: 2s - loss: 7.2805 - mean_absolute_error: 2.143680000/80000 [==============================] - 5s 57us/step - loss: 7.3334 - mean_absolute_error: 2.1518 - val_loss: 4.1336 - val_mean_absolute_error: 1.8176
Epoch 5/100
80000/80000 [==============================] - 5s 58us/step - loss: 7.2970 - mean_absolute_error: 2.1421 - val_loss: 4.1978 - val_mean_absolute_error: 1.8331
E

In [48]:
score =NCF_model.evaluate([Users1,Movies1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

20000/20000 [==============================] - 1s 30us/step
Test MSE: 4.155227939987182
Test MAE: 1.8234093242645264


### Size of Layer: 50

In [49]:
K_LATENT = 20
hidden_dim = 50
do = 0.5
ac = 'relu'
oprimaizer_name = 'adamax'
loss_function = 'mse'
activ_func ='relu'

NCF_model = get_ncf_model(max_userid,max_movieid,K_LATENT,hidden_dim,do,activ_func)
NCF_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()
callbacks_ncf = [EarlyStopping('val_loss', patience=5), ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncf_'+str(do)+'_'+str(K_LATENT)+'_'+str(hidden_dim)+'.h5', save_best_only=True)]
history = NCF_model.fit([Users, Movies], Ratings, nb_epoch=100,validation_data=([Users1,Movies1], Ratings1), validation_split=.1, verbose=1, callbacks=callbacks_ncf, batch_size = 32)

end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("dr...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  del sys.path[0]


Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 5s 67us/step - loss: 7.7441 - mean_absolute_error: 2.2877 - val_loss: 4.5346 - val_mean_absolute_error: 1.9091
Epoch 2/100
80000/80000 [==============================] - 5s 60us/step - loss: 7.3701 - mean_absolute_error: 2.1779 - val_loss: 4.2471 - val_mean_absolute_error: 1.8449
Epoch 3/100
80000/80000 [==============================] - 5s 60us/step - loss: 7.3105 - mean_absolute_error: 2.1540 - val_loss: 4.2110 - val_mean_absolute_error: 1.8364
Epoch 4/100
18016/80000 [=====>........................] - ETA: 3s - loss: 7.2462 - mean_absolute_error: 2.137480000/80000 [==============================] - 5s 60us/step - loss: 7.3080 - mean_absolute_error: 2.1490 - val_loss: 4.1461 - val_mean_absolute_error: 1.8215
Epoch 5/100
80000/80000 [==============================] - 5s 59us/step - loss: 7.2977 - mean_absolute_error: 2.1404 - val_loss: 4.1907 - val_mean_absolute_error: 1.8315
E

In [50]:
score =NCF_model.evaluate([Users1,Movies1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

20000/20000 [==============================] - 1s 30us/step
Test MSE: 4.136319258117676
Test MAE: 1.8218731609344483


### Size of Layer:  100

In [51]:
K_LATENT = 20
hidden_dim = 100
do = 0.5
ac = 'relu'
oprimaizer_name = 'adamax'
loss_function = 'mse'
activ_func ='relu'

NCF_model = get_ncf_model(max_userid,max_movieid,K_LATENT,hidden_dim,do,activ_func)
NCF_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()
callbacks_ncf = [EarlyStopping('val_loss', patience=5), ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncf_'+str(do)+'_'+str(K_LATENT)+'_'+str(hidden_dim)+'.h5', save_best_only=True)]
history = NCF_model.fit([Users, Movies], Ratings, nb_epoch=100,validation_data=([Users1,Movies1], Ratings1), validation_split=.1, verbose=1, callbacks=callbacks_ncf, batch_size = 32)

end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("dr...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  del sys.path[0]


Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 5s 69us/step - loss: 7.6548 - mean_absolute_error: 2.2535 - val_loss: 4.3267 - val_mean_absolute_error: 1.8629
Epoch 2/100
80000/80000 [==============================] - 5s 61us/step - loss: 7.3637 - mean_absolute_error: 2.1698 - val_loss: 4.2469 - val_mean_absolute_error: 1.8459
Epoch 3/100
80000/80000 [==============================] - 5s 61us/step - loss: 7.3084 - mean_absolute_error: 2.1509 - val_loss: 4.2403 - val_mean_absolute_error: 1.8443
Epoch 4/100
16736/80000 [=====>........................] - ETA: 3s - loss: 7.2515 - mean_absolute_error: 2.135180000/80000 [==============================] - 5s 61us/step - loss: 7.3114 - mean_absolute_error: 2.1478 - val_loss: 4.0926 - val_mean_absolute_error: 1.8094
Epoch 5/100
80000/80000 [==============================] - 5s 62us/step - loss: 7.3209 - mean_absolute_error: 2.1459 - val_loss: 4.2724 - val_mean_absolute_error: 1.8529
E

In [52]:
score =NCF_model.evaluate([Users1,Movies1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

20000/20000 [==============================] - 1s 31us/step
Test MSE: 4.126059072113037
Test MAE: 1.82011496257782


### Optimizer = sgd

In [175]:
K_LATENT = 20
hidden_dim = 100
do = 0.5
oprimaizer_name = 'sgd'
loss_function = 'mse'
activ_func ='relu'

NCF_model = get_ncf_model(max_userid,max_movieid,K_LATENT,hidden_dim,do,activ_func)
NCF_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()
callbacks_ncf = [EarlyStopping('val_loss', patience=5), ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncf_'+str(do)+'_'+str(K_LATENT)+'_'+str(hidden_dim)+'.h5', save_best_only=True)]
history = NCF_model.fit([Users, Movies], Ratings, nb_epoch=100,validation_data=([Users1,Movies1], Ratings1), validation_split=.1, verbose=1, callbacks=callbacks_ncf, batch_size = 32)


end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 18s 223us/step - loss: 7.5087 - mean_absolute_error: 2.2378 - val_loss: 4.5055 - val_mean_absolute_error: 1.8894
Epoch 2/100
80000/80000 [==============================] - 10s 121us/step - loss: 7.3793 - mean_absolute_error: 2.1900 - val_loss: 4.3805 - val_mean_absolute_error: 1.8686
Epoch 3/100
14304/80000 [====>.........................] - ETA: 7s - loss: 7.4121 - mean_absolute_error: 2.174880000/80000 [==============================] - 10s 129us/step - loss: 7.3753 - mean_absolute_error: 2.1664 - val_loss: 4.0254 - val_mean_absolute_error: 1.7876
Epoch 4/100
80000/80000 [==============================] - 10s 126us/step - loss: 7.3117 - mean_absolute_error: 2.1459 - val_loss: 4.1954 - val_mean_absolute_error: 1.8326
Epoch 5/100
33856/80000 [===========>..................] - ETA: 5s - loss: 7.3418 - mean_absolute_error: 2.148980000/80000 [==============================] - 11s 1

In [53]:
score =NCF_model.evaluate([Users1,Movies1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

20000/20000 [==============================] - 1s 30us/step
Test MSE: 4.126059072113037
Test MAE: 1.82011496257782


### Optimizer = adam


In [54]:
K_LATENT = 20
hidden_dim = 100
do = 0.5
oprimaizer_name = 'adam'
loss_function = 'mse'
activ_func ='relu'

NCF_model = get_ncf_model(max_userid,max_movieid,K_LATENT,hidden_dim,do,activ_func)
NCF_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()
callbacks_ncf = [EarlyStopping('val_loss', patience=5), ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncf_'+str(do)+'_'+str(K_LATENT)+'_'+str(hidden_dim)+'.h5', save_best_only=True)]
history = NCF_model.fit([Users, Movies], Ratings, nb_epoch=100,validation_data=([Users1,Movies1], Ratings1), validation_split=.1, verbose=1, callbacks=callbacks_ncf, batch_size = 32)

end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("dr...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if sys.path[0] == '':


Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 6s 73us/step - loss: 7.6052 - mean_absolute_error: 2.2394 - val_loss: 4.1331 - val_mean_absolute_error: 1.8154
Epoch 2/100
80000/80000 [==============================] - 5s 65us/step - loss: 7.3472 - mean_absolute_error: 2.1665 - val_loss: 4.0632 - val_mean_absolute_error: 1.8000
Epoch 3/100
80000/80000 [==============================] - 5s 65us/step - loss: 7.3666 - mean_absolute_error: 2.1627 - val_loss: 4.2775 - val_mean_absolute_error: 1.8525
Epoch 4/100
 8800/80000 [==>...........................] - ETA: 4s - loss: 7.2742 - mean_absolute_error: 2.133680000/80000 [==============================] - 5s 65us/step - loss: 7.2923 - mean_absolute_error: 2.1412 - val_loss: 4.1040 - val_mean_absolute_error: 1.8109
Epoch 5/100
80000/80000 [==============================] - 5s 65us/step - loss: 7.3080 - mean_absolute_error: 2.1420 - val_loss: 3.9857 - val_mean_absolute_error: 1.7822
E

In [55]:
score =NCF_model.evaluate([Users1,Movies1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

20000/20000 [==============================] - 1s 31us/step
Test MSE: 4.113116646957398
Test MAE: 1.8181356285095216


### Optimizer = Nadam

In [56]:
K_LATENT = 20
hidden_dim = 100
do = 0.5
oprimaizer_name = 'nadam'
loss_function = 'mse'
activ_func ='relu'

NCF_model = get_ncf_model(max_userid,max_movieid,K_LATENT,hidden_dim,do,activ_func)
NCF_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()
callbacks_ncf = [EarlyStopping('val_loss', patience=5), ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncf_'+str(do)+'_'+str(K_LATENT)+'_'+str(hidden_dim)+'.h5', save_best_only=True)]
history = NCF_model.fit([Users, Movies], Ratings, nb_epoch=100,validation_data=([Users1,Movies1], Ratings1), validation_split=.1, verbose=1, callbacks=callbacks_ncf, batch_size = 32)

end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("dr...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if sys.path[0] == '':


Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 6s 79us/step - loss: 7.5317 - mean_absolute_error: 2.2220 - val_loss: 4.2492 - val_mean_absolute_error: 1.8437
Epoch 2/100
80000/80000 [==============================] - 6s 70us/step - loss: 7.3060 - mean_absolute_error: 2.1525 - val_loss: 4.2300 - val_mean_absolute_error: 1.8429
Epoch 3/100
79872/80000 [============================>.] - ETA: 0s - loss: 7.3130 - mean_absolute_error: 2.146680000/80000 [==============================] - 6s 70us/step - loss: 7.3136 - mean_absolute_error: 2.1466 - val_loss: 4.1942 - val_mean_absolute_error: 1.8328
Epoch 4/100
80000/80000 [==============================] - 6s 70us/step - loss: 7.3362 - mean_absolute_error: 2.1471 - val_loss: 4.1208 - val_mean_absolute_error: 1.8154
Epoch 5/100
80000/80000 [==============================] - 6s 70us/step - loss: 7.3350 - mean_absolute_error: 2.1466 - val_loss: 4.1088 - val_mean_absolute_error: 1.8132
E

In [57]:
score =NCF_model.evaluate([Users1,Movies1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

20000/20000 [==============================] - 1s 31us/step
Test MSE: 4.184066261291504
Test MAE: 1.8374907342910767


### Optimizer = Adadelta

In [58]:
K_LATENT = 20
hidden_dim = 100
do = 0.5
oprimaizer_name = 'adadelta'
loss_function = 'mse'
activ_func ='relu'

NCF_model = get_ncf_model(max_userid,max_movieid,K_LATENT,hidden_dim,do,activ_func)
NCF_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()
callbacks_ncf = [EarlyStopping('val_loss', patience=5), ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncf_'+str(do)+'_'+str(K_LATENT)+'_'+str(hidden_dim)+'.h5', save_best_only=True)]
history = NCF_model.fit([Users, Movies], Ratings, nb_epoch=100,validation_data=([Users1,Movies1], Ratings1), validation_split=.1, verbose=1, callbacks=callbacks_ncf, batch_size = 32)

end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("dr...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if sys.path[0] == '':


Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 6s 78us/step - loss: 7.5267 - mean_absolute_error: 2.2247 - val_loss: 4.2710 - val_mean_absolute_error: 1.8481
Epoch 2/100
80000/80000 [==============================] - 6s 69us/step - loss: 7.3358 - mean_absolute_error: 2.1621 - val_loss: 4.2696 - val_mean_absolute_error: 1.8506
Epoch 3/100
80000/80000 [==============================] - 6s 69us/step - loss: 7.3378 - mean_absolute_error: 2.1545 - val_loss: 4.1682 - val_mean_absolute_error: 1.8284
Epoch 4/100
   32/80000 [..............................] - ETA: 14s - loss: 5.5243 - mean_absolute_error: 1.777880000/80000 [==============================] - 6s 69us/step - loss: 7.3256 - mean_absolute_error: 2.1483 - val_loss: 4.2337 - val_mean_absolute_error: 1.8427
Epoch 5/100
80000/80000 [==============================] - 6s 69us/step - loss: 7.3099 - mean_absolute_error: 2.1428 - val_loss: 4.2707 - val_mean_absolute_error: 1.8529


In [59]:
score =NCF_model.evaluate([Users1,Movies1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

20000/20000 [==============================] - 1s 31us/step
Test MSE: 4.151777981567383
Test MAE: 1.8270294820785522


### Function Loss = mean absolute error

In [60]:
K_LATENT = 20
hidden_dim = 100
do = 0.5
oprimaizer_name = 'nadam'
loss_function = 'mae'
activ_func ='relu'

NCF_model = get_ncf_model(max_userid,max_movieid,K_LATENT,hidden_dim,do,activ_func)
NCF_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()
callbacks_ncf = [EarlyStopping('val_loss', patience=5), ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncf_'+str(do)+'_'+str(K_LATENT)+'_'+str(hidden_dim)+'.h5', save_best_only=True)]
history = NCF_model.fit([Users, Movies], Ratings, nb_epoch=100,validation_data=([Users1,Movies1], Ratings1), validation_split=.1, verbose=1, callbacks=callbacks_ncf, batch_size = 32)

end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("dr...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if sys.path[0] == '':


Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 7s 81us/step - loss: 2.2074 - mean_absolute_error: 2.2074 - val_loss: 1.7564 - val_mean_absolute_error: 1.7564
Epoch 2/100
80000/80000 [==============================] - 6s 70us/step - loss: 2.1653 - mean_absolute_error: 2.1653 - val_loss: 1.7788 - val_mean_absolute_error: 1.7788
Epoch 3/100
79744/80000 [============================>.] - ETA: 0s - loss: 2.1453 - mean_absolute_error: 2.145380000/80000 [==============================] - 6s 70us/step - loss: 2.1452 - mean_absolute_error: 2.1452 - val_loss: 1.7432 - val_mean_absolute_error: 1.7432
Epoch 4/100
80000/80000 [==============================] - 6s 70us/step - loss: 2.1341 - mean_absolute_error: 2.1341 - val_loss: 1.7951 - val_mean_absolute_error: 1.7951
Epoch 5/100
80000/80000 [==============================] - 6s 71us/step - loss: 2.1363 - mean_absolute_error: 2.1363 - val_loss: 1.7890 - val_mean_absolute_error: 1.7890
E

In [61]:
score =NCF_model.evaluate([Users1,Movies1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

20000/20000 [==============================] - 1s 31us/step
Test MSE: 1.7840576414108276
Test MAE: 1.7840576414108276


### Function Loss =mean squared error

In [62]:
K_LATENT = 20
hidden_dim = 100
do = 0.5
oprimaizer_name = 'nadam'
loss_function = 'mse'
activ_func ='relu'

NCF_model = get_ncf_model(max_userid,max_movieid,K_LATENT,hidden_dim,do,activ_func)
NCF_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()
callbacks_ncf = [EarlyStopping('val_loss', patience=5), ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncf_'+str(do)+'_'+str(K_LATENT)+'_'+str(hidden_dim)+'.h5', save_best_only=True)]
history = NCF_model.fit([Users, Movies], Ratings, nb_epoch=100,validation_data=([Users1,Movies1], Ratings1), validation_split=.1, verbose=1, callbacks=callbacks_ncf, batch_size = 32)

end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("dr...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if sys.path[0] == '':


Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 7s 82us/step - loss: 7.5195 - mean_absolute_error: 2.2189 - val_loss: 4.1183 - val_mean_absolute_error: 1.8119
Epoch 2/100
80000/80000 [==============================] - 6s 71us/step - loss: 7.3387 - mean_absolute_error: 2.1602 - val_loss: 4.2292 - val_mean_absolute_error: 1.8411
Epoch 3/100
78720/80000 [============================>.] - ETA: 0s - loss: 7.2423 - mean_absolute_error: 2.134280000/80000 [==============================] - 6s 70us/step - loss: 7.2451 - mean_absolute_error: 2.1347 - val_loss: 4.1115 - val_mean_absolute_error: 1.8129
Epoch 4/100
80000/80000 [==============================] - 6s 71us/step - loss: 7.2677 - mean_absolute_error: 2.1343 - val_loss: 4.1941 - val_mean_absolute_error: 1.8306
Epoch 5/100
80000/80000 [==============================] - 6s 71us/step - loss: 7.3219 - mean_absolute_error: 2.1439 - val_loss: 4.3846 - val_mean_absolute_error: 1.8746
E

In [63]:
score =NCF_model.evaluate([Users1,Movies1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

20000/20000 [==============================] - 1s 32us/step
Test MSE: 4.107815170669555
Test MAE: 1.8211423374176026


### Function Loss = mean_absolute_percentage_error

In [64]:
K_LATENT = 20
hidden_dim = 100
do = 0.5
oprimaizer_name = 'nadam'
loss_function = 'mean_absolute_percentage_error'
activ_func ='relu'

NCF_model = get_ncf_model(max_userid,max_movieid,K_LATENT,hidden_dim,do,activ_func)
NCF_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()
callbacks_ncf = [EarlyStopping('val_loss', patience=5), ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncf_'+str(do)+'_'+str(K_LATENT)+'_'+str(hidden_dim)+'.h5', save_best_only=True)]
history = NCF_model.fit([Users, Movies], Ratings, nb_epoch=100,validation_data=([Users1,Movies1], Ratings1), validation_split=.1, verbose=1, callbacks=callbacks_ncf, batch_size = 32)

end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("dr...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if sys.path[0] == '':


Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 7s 83us/step - loss: 66.5913 - mean_absolute_error: 2.2604 - val_loss: 53.5828 - val_mean_absolute_error: 2.0113
Epoch 2/100
80000/80000 [==============================] - 6s 71us/step - loss: 64.6890 - mean_absolute_error: 2.2019 - val_loss: 50.9401 - val_mean_absolute_error: 1.9170
Epoch 3/100
75680/80000 [===========================>..] - ETA: 0s - loss: 64.4663 - mean_absolute_error: 2.190580000/80000 [==============================] - 6s 71us/step - loss: 64.4752 - mean_absolute_error: 2.1912 - val_loss: 54.8570 - val_mean_absolute_error: 2.0620
Epoch 4/100
80000/80000 [==============================] - 6s 71us/step - loss: 64.3062 - mean_absolute_error: 2.1850 - val_loss: 52.8141 - val_mean_absolute_error: 1.9729
Epoch 5/100
80000/80000 [==============================] - 6s 71us/step - loss: 64.0132 - mean_absolute_error: 2.1697 - val_loss: 51.4745 - val_mean_absolute_erro

In [65]:
score =NCF_model.evaluate([Users1,Movies1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

20000/20000 [==============================] - 1s 32us/step
Test MSE: 52.329094750976566
Test MAE: 1.970181530570984


### Activation Function = relu

In [66]:
K_LATENT = 20
hidden_dim = 100
do = 0.5
oprimaizer_name = 'nadam'
loss_function = 'mae'
activ_func ='relu'

NCF_model = get_ncf_model(max_userid,max_movieid,K_LATENT,hidden_dim,do,activ_func)
NCF_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()
callbacks_ncf = [EarlyStopping('val_loss', patience=5), ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncf_'+str(do)+'_'+str(K_LATENT)+'_'+str(hidden_dim)+'.h5', save_best_only=True)]
history = NCF_model.fit([Users, Movies], Ratings, nb_epoch=100,validation_data=([Users1,Movies1], Ratings1), validation_split=.1, verbose=1, callbacks=callbacks_ncf, batch_size = 32)

end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("dr...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if sys.path[0] == '':


Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 7s 84us/step - loss: 2.2263 - mean_absolute_error: 2.2263 - val_loss: 1.7519 - val_mean_absolute_error: 1.7519
Epoch 2/100
80000/80000 [==============================] - 6s 71us/step - loss: 2.1659 - mean_absolute_error: 2.1659 - val_loss: 1.7864 - val_mean_absolute_error: 1.7864
Epoch 3/100
76576/80000 [===========================>..] - ETA: 0s - loss: 2.1411 - mean_absolute_error: 2.141180000/80000 [==============================] - 6s 71us/step - loss: 2.1405 - mean_absolute_error: 2.1405 - val_loss: 1.8066 - val_mean_absolute_error: 1.8066
Epoch 4/100
80000/80000 [==============================] - 6s 71us/step - loss: 2.1404 - mean_absolute_error: 2.1404 - val_loss: 1.7603 - val_mean_absolute_error: 1.7603
Epoch 5/100
80000/80000 [==============================] - 6s 71us/step - loss: 2.1329 - mean_absolute_error: 2.1329 - val_loss: 1.7958 - val_mean_absolute_error: 1.7958
E

In [67]:
score =NCF_model.evaluate([Users1,Movies1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

20000/20000 [==============================] - 1s 32us/step
Test MSE: 1.8126866855621337
Test MAE: 1.8126866855621337


### Activation Function = tanh

In [68]:
K_LATENT = 20
hidden_dim = 100
do = 0.5
oprimaizer_name = 'nadam'
loss_function = 'mae'
activ_func ='tanh'

NCF_model = get_ncf_model(max_userid,max_movieid,K_LATENT,hidden_dim,do,activ_func)
NCF_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()
callbacks_ncf = [EarlyStopping('val_loss', patience=5), ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncf_'+str(do)+'_'+str(K_LATENT)+'_'+str(hidden_dim)+'.h5', save_best_only=True)]
history = NCF_model.fit([Users, Movies], Ratings, nb_epoch=100,validation_data=([Users1,Movies1], Ratings1), validation_split=.1, verbose=1, callbacks=callbacks_ncf, batch_size = 32)

end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("dr...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if sys.path[0] == '':


Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 7s 85us/step - loss: 2.6007 - mean_absolute_error: 2.6007 - val_loss: 2.5362 - val_mean_absolute_error: 2.5362
Epoch 2/100
80000/80000 [==============================] - 6s 71us/step - loss: 2.5903 - mean_absolute_error: 2.5903 - val_loss: 2.5359 - val_mean_absolute_error: 2.5359
Epoch 3/100
77600/80000 [============================>.] - ETA: 0s - loss: 2.5838 - mean_absolute_error: 2.583880000/80000 [==============================] - 6s 71us/step - loss: 2.5843 - mean_absolute_error: 2.5843 - val_loss: 2.5359 - val_mean_absolute_error: 2.5359
Epoch 4/100
80000/80000 [==============================] - 6s 70us/step - loss: 2.5898 - mean_absolute_error: 2.5898 - val_loss: 2.5359 - val_mean_absolute_error: 2.5359
Epoch 5/100
80000/80000 [==============================] - 6s 71us/step - loss: 2.5860 - mean_absolute_error: 2.5860 - val_loss: 2.5359 - val_mean_absolute_error: 2.5359
E

In [69]:
score =NCF_model.evaluate([Users1,Movies1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

20000/20000 [==============================] - 1s 32us/step
Test MSE: 2.538334713554382
Test MAE: 2.538334713554382


### Activation Function = linear

In [70]:
K_LATENT = 20
hidden_dim = 100
do = 0.5
oprimaizer_name = 'nadam'
loss_function = 'mae'
activ_func ='linear'

NCF_model = get_ncf_model(max_userid,max_movieid,K_LATENT,hidden_dim,do,activ_func)
NCF_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()
callbacks_ncf = [EarlyStopping('val_loss', patience=5), ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncf_'+str(do)+'_'+str(K_LATENT)+'_'+str(hidden_dim)+'.h5', save_best_only=True)]
history = NCF_model.fit([Users, Movies], Ratings, nb_epoch=100,validation_data=([Users1,Movies1], Ratings1), validation_split=.1, verbose=1, callbacks=callbacks_ncf, batch_size = 32)

end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("dr...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if sys.path[0] == '':


Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 7s 84us/step - loss: 2.2027 - mean_absolute_error: 2.2027 - val_loss: 1.7831 - val_mean_absolute_error: 1.7831
Epoch 2/100
80000/80000 [==============================] - 6s 70us/step - loss: 2.1624 - mean_absolute_error: 2.1624 - val_loss: 1.7611 - val_mean_absolute_error: 1.7611
Epoch 3/100
79840/80000 [============================>.] - ETA: 0s - loss: 2.1444 - mean_absolute_error: 2.144480000/80000 [==============================] - 6s 70us/step - loss: 2.1446 - mean_absolute_error: 2.1446 - val_loss: 1.7871 - val_mean_absolute_error: 1.7871
Epoch 4/100
80000/80000 [==============================] - 6s 71us/step - loss: 2.1442 - mean_absolute_error: 2.1442 - val_loss: 1.7775 - val_mean_absolute_error: 1.7775
Epoch 5/100
80000/80000 [==============================] - 6s 70us/step - loss: 2.1435 - mean_absolute_error: 2.1435 - val_loss: 1.8052 - val_mean_absolute_error: 1.8052
E

In [71]:
score =NCF_model.evaluate([Users1,Movies1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

20000/20000 [==============================] - 1s 32us/step
Test MSE: 1.766087169647217
Test MAE: 1.766087169647217


### Activation Function = selu

In [72]:
K_LATENT = 20
hidden_dim = 100
do = 0.5
oprimaizer_name = 'nadam'
loss_function = 'mae'
activ_func ='selu'

NCF_model = get_ncf_model(max_userid,max_movieid,K_LATENT,hidden_dim,do,activ_func)
NCF_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()
callbacks_ncf = [EarlyStopping('val_loss', patience=5), ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncf_'+str(do)+'_'+str(K_LATENT)+'_'+str(hidden_dim)+'.h5', save_best_only=True)]
history = NCF_model.fit([Users, Movies], Ratings, nb_epoch=100,validation_data=([Users1,Movies1], Ratings1), validation_split=.1, verbose=1, callbacks=callbacks_ncf, batch_size = 32)

end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("dr...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if sys.path[0] == '':


Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 7s 89us/step - loss: 2.1927 - mean_absolute_error: 2.1927 - val_loss: 1.7881 - val_mean_absolute_error: 1.7881
Epoch 2/100
80000/80000 [==============================] - 6s 75us/step - loss: 2.1757 - mean_absolute_error: 2.1757 - val_loss: 1.7611 - val_mean_absolute_error: 1.7611
Epoch 3/100
70944/80000 [=========================>....] - ETA: 0s - loss: 2.1482 - mean_absolute_error: 2.148280000/80000 [==============================] - 6s 74us/step - loss: 2.1453 - mean_absolute_error: 2.1453 - val_loss: 1.8179 - val_mean_absolute_error: 1.8179
Epoch 4/100
80000/80000 [==============================] - 6s 75us/step - loss: 2.1465 - mean_absolute_error: 2.1465 - val_loss: 1.7783 - val_mean_absolute_error: 1.7783
Epoch 5/100
80000/80000 [==============================] - 6s 74us/step - loss: 2.1395 - mean_absolute_error: 2.1395 - val_loss: 1.8014 - val_mean_absolute_error: 1.8014
E

In [73]:
score =NCF_model.evaluate([Users1,Movies1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

20000/20000 [==============================] - 1s 34us/step
Test MSE: 1.7997085922241212
Test MAE: 1.7997085922241212


### Dropout = 0.1


In [74]:
K_LATENT = 20
hidden_dim = 100
do = 0.1
oprimaizer_name = 'nadam'
loss_function = 'mae'
activ_func ='relu'

NCF_model = get_ncf_model(max_userid,max_movieid,K_LATENT,hidden_dim,do,activ_func)
NCF_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()
callbacks_ncf = [EarlyStopping('val_loss', patience=5), ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncf_'+str(do)+'_'+str(K_LATENT)+'_'+str(hidden_dim)+'.h5', save_best_only=True)]
history = NCF_model.fit([Users, Movies], Ratings, nb_epoch=100,validation_data=([Users1,Movies1], Ratings1), validation_split=.1, verbose=1, callbacks=callbacks_ncf, batch_size = 32)

end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("dr...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if sys.path[0] == '':


Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 8s 94us/step - loss: 1.1003 - mean_absolute_error: 1.1003 - val_loss: 0.8404 - val_mean_absolute_error: 0.8404
Epoch 2/100
80000/80000 [==============================] - 6s 71us/step - loss: 1.0208 - mean_absolute_error: 1.0208 - val_loss: 0.8223 - val_mean_absolute_error: 0.8223
Epoch 3/100
75072/80000 [===========================>..] - ETA: 0s - loss: 1.0171 - mean_absolute_error: 1.017180000/80000 [==============================] - 6s 71us/step - loss: 1.0167 - mean_absolute_error: 1.0167 - val_loss: 0.8137 - val_mean_absolute_error: 0.8137
Epoch 4/100
80000/80000 [==============================] - 6s 72us/step - loss: 1.0101 - mean_absolute_error: 1.0101 - val_loss: 0.8464 - val_mean_absolute_error: 0.8464
Epoch 5/100
80000/80000 [==============================] - 6s 71us/step - loss: 1.0023 - mean_absolute_error: 1.0023 - val_loss: 0.8464 - val_mean_absolute_error: 0.8464
E

In [75]:
score =NCF_model.evaluate([Users1,Movies1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

20000/20000 [==============================] - 1s 33us/step
Test MSE: 0.7967103713989258
Test MAE: 0.7967103713989258


### Dropout = 0.5 


In [76]:
K_LATENT = 20
hidden_dim = 100
do = 0.5
oprimaizer_name = 'nadam'
loss_function = 'mae'
activ_func ='relu'

NCF_model = get_ncf_model(max_userid,max_movieid,K_LATENT,hidden_dim,do,activ_func)
NCF_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()
callbacks_ncf = [EarlyStopping('val_loss', patience=5), ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncf_'+str(do)+'_'+str(K_LATENT)+'_'+str(hidden_dim)+'.h5', save_best_only=True)]
history = NCF_model.fit([Users, Movies], Ratings, nb_epoch=100,validation_data=([Users1,Movies1], Ratings1), validation_split=.1, verbose=1, callbacks=callbacks_ncf, batch_size = 32)

end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("dr...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if sys.path[0] == '':


Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 7s 89us/step - loss: 2.2248 - mean_absolute_error: 2.2248 - val_loss: 1.7713 - val_mean_absolute_error: 1.7713
Epoch 2/100
80000/80000 [==============================] - 6s 72us/step - loss: 2.1638 - mean_absolute_error: 2.1638 - val_loss: 1.8757 - val_mean_absolute_error: 1.8757
Epoch 3/100
74880/80000 [===========================>..] - ETA: 0s - loss: 2.1449 - mean_absolute_error: 2.144980000/80000 [==============================] - 6s 72us/step - loss: 2.1446 - mean_absolute_error: 2.1446 - val_loss: 1.7782 - val_mean_absolute_error: 1.7782
Epoch 4/100
80000/80000 [==============================] - 6s 72us/step - loss: 2.1380 - mean_absolute_error: 2.1380 - val_loss: 1.7863 - val_mean_absolute_error: 1.7863
Epoch 5/100
80000/80000 [==============================] - 6s 72us/step - loss: 2.1348 - mean_absolute_error: 2.1348 - val_loss: 1.8550 - val_mean_absolute_error: 1.8550
E

In [77]:
score =NCF_model.evaluate([Users1,Movies1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

20000/20000 [==============================] - 1s 32us/step
Test MSE: 1.8371835897445679
Test MAE: 1.8371835897445679


In [108]:
K_LATENT = 20
hidden_dim = 100
do = 0.1
oprimaizer_name = 'adam'
loss_function = 'mae'
activ_func ='linear'

NCF_model = get_ncf_model(max_userid,max_movieid,K_LATENT,hidden_dim,do,activ_func)
NCF_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()
callbacks_ncf = [EarlyStopping('val_loss', patience=5), ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncf_'+str(do)+'_'+str(K_LATENT)+'_'+str(hidden_dim)+'.h5', save_best_only=True)]
history = NCF_model.fit([Users, Movies], Ratings, nb_epoch=100,validation_data=([Users1,Movies1], Ratings1), validation_split=.1, verbose=1, callbacks=callbacks_ncf, batch_size = 32)

end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

Train on 64000 samples, validate on 16000 samples
Epoch 1/100
64000/64000 [==============================] - 7s 103us/step - loss: 1.2530 - mean_absolute_error: 1.2530 - val_loss: 0.9445 - val_mean_absolute_error: 0.9445
Epoch 2/100
64000/64000 [==============================] - 5s 75us/step - loss: 1.1771 - mean_absolute_error: 1.1771 - val_loss: 0.9522 - val_mean_absolute_error: 0.9522
Epoch 3/100
64000/64000 [==============================] - 5s 74us/step - loss: 1.1762 - mean_absolute_error: 1.1762 - val_loss: 0.9584 - val_mean_absolute_error: 0.9584
Epoch 4/100
16384/64000 [======>.......................] - ETA: 3s - loss: 1.1790 - mean_absolute_error: 1.179064000/64000 [==============================] - 5s 73us/step - loss: 1.1660 - mean_absolute_error: 1.1660 - val_loss: 0.9522 - val_mean_absolute_error: 0.9522
Epoch 5/100
64000/64000 [==============================] - 5s 73us/step - loss: 1.1644 - mean_absolute_error: 1.1644 - val_loss: 0.9583 - val_mean_absolute_error: 0.9583


In [109]:
score =NCF_model.evaluate([Users1,Movies1], Ratings1,verbose=1)
print('Test MAE:', score[1])

16000/16000 [==============================] - 1s 36us/step
Test MAE: 0.9512899843454361


### End of part 2  - all result sum up are in table in the pdf.

## Part 3 - Age and Gender as Extra feature


In [78]:
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')



# reading data and splitting in 80-20


%matplotlib inline
u_cols = ['user_id', 'age', 'gender', 'occupation', 'zip_code']
users = pd.read_csv('u.user', sep='|', names=u_cols,
                    encoding='latin-1', parse_dates=True) 

r_cols = ['user_id', 'item_id', 'rating', 'timestamp']
ratings = pd.read_csv('u1.base', sep='\t', names=r_cols,
                      encoding='latin-1')

m_cols = ['item_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv('u.item', sep='|', names=m_cols, usecols=range(5),
                     encoding='latin-1')





# df.drop(df.columns[[3,4,7]], axis=1, inplace=True)
ratings.drop("timestamp", inplace = True, axis = 1 ) 
movies.drop(movies.columns[[1,2,3,4]], inplace = True, axis = 1 )
users.drop(users.columns[[3,4]], inplace = True, axis = 1 )

movie_ratings = pd.merge(movies, ratings)
movie_ratings_user = pd.merge(movie_ratings, users)

movie_ratings_user.head()





,item_id,user_id,rating,age,gender
0,1,1,5,24,M
1,2,1,3,24,M
2,3,1,4,24,M
3,4,1,3,24,M
4,5,1,3,24,M


In [79]:
max_userid = movie_ratings_user['user_id'].drop_duplicates().max()
max_movieid = movie_ratings_user['item_id'].drop_duplicates().max()
movie_ratings_user['user_emb_id'] = movie_ratings_user['user_id'] - 1
movie_ratings_user['movie_emb_id'] = movie_ratings_user['item_id'] - 1
print(str(len(movie_ratings_user))+' ratings loaded')

movie_ratings_user.head()


80000 ratings loaded


,item_id,user_id,rating,age,gender,user_emb_id,movie_emb_id
0,1,1,5,24,M,0,0
1,2,1,3,24,M,0,1
2,3,1,4,24,M,0,2
3,4,1,3,24,M,0,3
4,5,1,3,24,M,0,4


In [80]:
RATINGS_GEN_CSV_FILE = 'u_genr.data'
movie_ratings_user.to_csv(RATINGS_GEN_CSV_FILE, 
               sep='\t', 
               header=True, 
               encoding='latin-1', 
               columns=['user_id', 'item_id', 'rating', 'timestamp', 'gender','user_emb_id', 'movie_emb_id'])
print('Saved to '+RATINGS_GEN_CSV_FILE)

Saved to u_genr.data


In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

UsersT = movie_ratings_user['user_emb_id'].values
MoviesT = movie_ratings_user['movie_emb_id'].values
RatingsT = movie_ratings_user['rating'].values
GenderT = movie_ratings_user['gender'].astype('category').cat.codes.values
AgeT = movie_ratings_user['age'].astype('category').cat.codes.values



Users, Users1 = train_test_split(UsersT, test_size=0.2)
Movies, Movies1 = train_test_split(MoviesT, test_size=0.2)
Ratings , Ratings1 = train_test_split(RatingsT, test_size=0.2)
Gender, Gender1 = train_test_split(GenderT, test_size=0.2)
Age , Age1 = train_test_split(AgeT, test_size=0.2)

In [0]:
#CNF and gender as feature
def get_ncf_model_gender(num_users, num_items, latent_dim,hidden_dim,do):
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')
    gender_input = Input(shape=(1,), dtype='float32', name = 'gender_input')

    MF_Embedding_User = Embedding(input_dim = num_users, output_dim = latent_dim, name = 'user_embedding', input_length=1)
    MF_Embedding_Item = Embedding(input_dim = num_items, output_dim = latent_dim, name = 'item_embedding', input_length=1)   
    
    # Crucial to flatten an embedding vector!
    user_latent = Flatten()(MF_Embedding_User(user_input))
    item_latent = Flatten()(MF_Embedding_Item(item_input))
    
    # Concat user and item embeddings with gender
    conc = Concatenate()([user_latent, item_latent,gender_input])
    drop = Dropout(do)(conc)
    hid1 = Dense(hidden_dim, activation='relu')(conc)
    drop2  = Dropout(do)(hid1)
    hid2 = Dense(10,activation='relu')(drop2)
    drop3  = Dropout(do)(hid2)
    prediction = Dense(1, activation='relu', kernel_initializer='lecun_uniform', name = 'prediction')(drop2)
    
    
    model = Model(input=[user_input, item_input,gender_input], output=prediction)

    return model
  
  
  
#CNF and age as feature
def get_ncf_model_age(num_users, num_items, latent_dim,hidden_dim,do):
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')
    age_input = Input(shape=(1,), dtype='float32', name = 'age_input')

    MF_Embedding_User = Embedding(input_dim = num_users, output_dim = latent_dim, name = 'user_embedding', input_length=1)
    MF_Embedding_Item = Embedding(input_dim = num_items, output_dim = latent_dim, name = 'item_embedding', input_length=1)   
    
    # Crucial to flatten an embedding vector!
    user_latent = Flatten()(MF_Embedding_User(user_input))
    item_latent = Flatten()(MF_Embedding_Item(item_input))
    
    # Concat user and item embeddings with age
    conc = Concatenate()([user_latent, item_latent,age_input])
    drop = Dropout(do)(conc)
    hid1 = Dense(hidden_dim, activation='relu')(conc)
    drop2  = Dropout(do)(hid1)
    #hid2 = Dense(10,activation='relu')(drop2)
    #drop3  = Dropout(do)(hid2)
    prediction = Dense(1, activation='relu', kernel_initializer='lecun_uniform', name = 'prediction')(drop2)
    
    
    model = Model(input=[user_input, item_input,age_input], output=prediction)

    return model

### NCF + genger as feature

In [83]:
K_LATENT = 20
hidden_dim = 100
do = 0.1
oprimaizer_name = 'nadam'
loss_function = 'mae'
activ_func ='relu'


Gender_NCFG_model = get_ncf_model_gender(max_userid,max_movieid,K_LATENT,hidden_dim,do)
Gender_NCFG_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()

callbacks_ncfgender = [EarlyStopping('val_loss', patience=20), 
             ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncfg_'+str(do)+'_'+str(K_LATENT)+'.h5', save_best_only=True)]
history = Gender_NCFG_model.fit([Users, Movies,Gender], Ratings, nb_epoch=100, validation_data=([Users1,Movies1,Gender1], Ratings1),validation_split=.2, verbose=1, callbacks=callbacks_ncfgender, batch_size = 32)


end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

Train on 64000 samples, validate on 16000 samples
Epoch 1/100
64000/64000 [==============================] - 6s 94us/step - loss: 0.9661 - mean_absolute_error: 0.9661 - val_loss: 0.9268 - val_mean_absolute_error: 0.9268
Epoch 2/100
64000/64000 [==============================] - 5s 74us/step - loss: 0.9185 - mean_absolute_error: 0.9185 - val_loss: 0.9243 - val_mean_absolute_error: 0.9243
Epoch 3/100
64000/64000 [==============================] - 5s 74us/step - loss: 0.9095 - mean_absolute_error: 0.9095 - val_loss: 0.9376 - val_mean_absolute_error: 0.9376
Epoch 4/100
16768/64000 [======>.......................] - ETA: 3s - loss: 0.8904 - mean_absolute_error: 0.890464000/64000 [==============================] - 5s 74us/step - loss: 0.9020 - mean_absolute_error: 0.9020 - val_loss: 0.9335 - val_mean_absolute_error: 0.9335
Epoch 5/100
64000/64000 [==============================] - 5s 74us/step - loss: 0.8930 - mean_absolute_error: 0.8930 - val_loss: 0.9369 - val_mean_absolute_error: 0.9369
E

In [84]:
score =Gender_NCFG_model.evaluate([Users1,Movies1,Gender1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

16000/16000 [==============================] - 1s 34us/step
Test MSE: 1.013292018175125
Test MAE: 1.013292018175125



### drop-out = 0.*5*



In [85]:
K_LATENT = 20
hidden_dim = 100
do = 0.5
oprimaizer_name = 'nadam'
loss_function = 'mae'
activ_func ='relu'


Gender_NCFG_model = get_ncf_model_gender(max_userid,max_movieid,K_LATENT,hidden_dim,do)
Gender_NCFG_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()

callbacks_ncfgender = [EarlyStopping('val_loss', patience=20), 
             ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncfg_'+str(do)+'_'+str(K_LATENT)+'.h5', save_best_only=True)]
history = Gender_NCFG_model.fit([Users, Movies,Gender], Ratings, nb_epoch=100,validation_data=([Users1,Movies1,Gender1], Ratings1), validation_split=.2, verbose=1, callbacks=callbacks_ncfgender, batch_size = 32)


end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

Train on 64000 samples, validate on 16000 samples
Epoch 1/100
64000/64000 [==============================] - 6s 95us/step - loss: 1.0307 - mean_absolute_error: 1.0307 - val_loss: 0.9410 - val_mean_absolute_error: 0.9410
Epoch 2/100
64000/64000 [==============================] - 5s 73us/step - loss: 0.9526 - mean_absolute_error: 0.9526 - val_loss: 0.9471 - val_mean_absolute_error: 0.9471
Epoch 3/100
64000/64000 [==============================] - 5s 73us/step - loss: 0.9352 - mean_absolute_error: 0.9352 - val_loss: 0.9362 - val_mean_absolute_error: 0.9362
Epoch 4/100
17888/64000 [=======>......................] - ETA: 3s - loss: 0.9150 - mean_absolute_error: 0.915064000/64000 [==============================] - 5s 73us/step - loss: 0.9209 - mean_absolute_error: 0.9209 - val_loss: 0.9278 - val_mean_absolute_error: 0.9278
Epoch 5/100
64000/64000 [==============================] - 5s 73us/step - loss: 0.9096 - mean_absolute_error: 0.9096 - val_loss: 0.9340 - val_mean_absolute_error: 0.9340
E

In [86]:
score =Gender_NCFG_model.evaluate([Users1,Movies1,Gender1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

16000/16000 [==============================] - 1s 34us/step
Test MSE: 0.9618893321752549
Test MAE: 0.9618893321752549


### optimezer = adamax

In [87]:
K_LATENT = 20
hidden_dim = 100
do = 0.5
oprimaizer_name = 'adamax'
loss_function = 'mae'
activ_func ='relu'


Gender_NCFG_model = get_ncf_model_gender(max_userid,max_movieid,K_LATENT,hidden_dim,do)
Gender_NCFG_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()

callbacks_ncfgender = [EarlyStopping('val_loss', patience=20), 
             ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncfg_'+str(do)+'_'+str(K_LATENT)+'.h5', save_best_only=True)]
history = Gender_NCFG_model.fit([Users, Movies,Gender], Ratings, nb_epoch=100,validation_data=([Users1,Movies1,Gender1], Ratings1), validation_split=.2, verbose=1, callbacks=callbacks_ncfgender, batch_size = 32)


end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

Train on 64000 samples, validate on 16000 samples
Epoch 1/100
64000/64000 [==============================] - 6s 88us/step - loss: 1.0740 - mean_absolute_error: 1.0740 - val_loss: 0.9350 - val_mean_absolute_error: 0.9350
Epoch 2/100
64000/64000 [==============================] - 4s 65us/step - loss: 0.9567 - mean_absolute_error: 0.9567 - val_loss: 0.9347 - val_mean_absolute_error: 0.9347
Epoch 3/100
64000/64000 [==============================] - 4s 65us/step - loss: 0.9462 - mean_absolute_error: 0.9462 - val_loss: 0.9262 - val_mean_absolute_error: 0.9262
Epoch 4/100
32256/64000 [==============>...............] - ETA: 1s - loss: 0.9386 - mean_absolute_error: 0.938664000/64000 [==============================] - 4s 66us/step - loss: 0.9377 - mean_absolute_error: 0.9377 - val_loss: 0.9324 - val_mean_absolute_error: 0.9324
Epoch 5/100
64000/64000 [==============================] - 4s 66us/step - loss: 0.9320 - mean_absolute_error: 0.9320 - val_loss: 0.9369 - val_mean_absolute_error: 0.9369
E

In [88]:
score =Gender_NCFG_model.evaluate([Users1,Movies1,Gender1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

16000/16000 [==============================] - 1s 34us/step
Test MSE: 0.9246920326948166
Test MAE: 0.9246920326948166


### loss function = mse

In [89]:
K_LATENT = 20
hidden_dim = 100
do = 0.5
oprimaizer_name = 'nadam'
loss_function = 'mse'
activ_func ='relu'


Gender_NCFG_model = get_ncf_model_gender(max_userid,max_movieid,K_LATENT,hidden_dim,do)
Gender_NCFG_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
Gender_NCFG_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         (None, 1)            0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 20)        18860       user_input[0][0]                 
__________________________________________________________________________________________________
item_embedding (Embedding)      (None, 1, 20)        33640       item_input[0][0]                 
__________________________________________________________________________________________________
flatten_51

In [90]:
K_LATENT = 20
hidden_dim = 100
do = 0.5
oprimaizer_name = 'nadam'
loss_function = 'mse'
activ_func ='relu'


Gender_NCFG_model = get_ncf_model_gender(max_userid,max_movieid,K_LATENT,hidden_dim,do)
Gender_NCFG_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()

callbacks_ncfgender = [EarlyStopping('val_loss', patience=20), 
             ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncfg_'+str(do)+'_'+str(K_LATENT)+'.h5', save_best_only=True)]
history = Gender_NCFG_model.fit([Users, Movies,Gender], Ratings, nb_epoch=100,validation_data=([Users1,Movies1,Gender1], Ratings1), validation_split=.2, verbose=1, callbacks=callbacks_ncfgender, batch_size = 32)


end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

Train on 64000 samples, validate on 16000 samples
Epoch 1/100
64000/64000 [==============================] - 6s 99us/step - loss: 1.6690 - mean_absolute_error: 1.0326 - val_loss: 1.2757 - val_mean_absolute_error: 0.9520
Epoch 2/100
64000/64000 [==============================] - 5s 74us/step - loss: 1.3783 - mean_absolute_error: 0.9581 - val_loss: 1.2632 - val_mean_absolute_error: 0.9400
Epoch 3/100
64000/64000 [==============================] - 5s 74us/step - loss: 1.3134 - mean_absolute_error: 0.9381 - val_loss: 1.2692 - val_mean_absolute_error: 0.9413
Epoch 4/100
16224/64000 [======>.......................] - ETA: 3s - loss: 1.2617 - mean_absolute_error: 0.920164000/64000 [==============================] - 5s 74us/step - loss: 1.2743 - mean_absolute_error: 0.9269 - val_loss: 1.2738 - val_mean_absolute_error: 0.9410
Epoch 5/100
64000/64000 [==============================] - 5s 74us/step - loss: 1.2498 - mean_absolute_error: 0.9204 - val_loss: 1.2841 - val_mean_absolute_error: 0.9485
E

In [91]:
score =Gender_NCFG_model.evaluate([Users1,Movies1,Gender1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

16000/16000 [==============================] - 1s 34us/step
Test MSE: 1.3917988891601563
Test MAE: 0.9613025593757629


### activation function = selu and layer size = 50

In [92]:
K_LATENT = 20
hidden_dim = 50
do = 0.1
oprimaizer_name = 'nadam'
loss_function = 'mse'
activ_func ='selu'


Gender_NCFG_model = get_ncf_model_gender(max_userid,max_movieid,K_LATENT,hidden_dim,do)
Gender_NCFG_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()

callbacks_ncfgender = [EarlyStopping('val_loss', patience=20), 
             ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncfg_'+str(do)+'_'+str(K_LATENT)+'.h5', save_best_only=True)]
history = Gender_NCFG_model.fit([Users, Movies,Gender], Ratings, nb_epoch=100,validation_data=([Users1,Movies1,Gender1], Ratings1), validation_split=.2, verbose=1, callbacks=callbacks_ncfgender, batch_size = 32)


end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

Train on 64000 samples, validate on 16000 samples
Epoch 1/100
64000/64000 [==============================] - 6s 96us/step - loss: 1.5600 - mean_absolute_error: 1.0033 - val_loss: 1.2811 - val_mean_absolute_error: 0.9281
Epoch 2/100
64000/64000 [==============================] - 5s 72us/step - loss: 1.2972 - mean_absolute_error: 0.9365 - val_loss: 1.2667 - val_mean_absolute_error: 0.9423
Epoch 3/100
64000/64000 [==============================] - 5s 72us/step - loss: 1.2756 - mean_absolute_error: 0.9265 - val_loss: 1.2830 - val_mean_absolute_error: 0.9462
Epoch 4/100
19872/64000 [========>.....................] - ETA: 2s - loss: 1.2558 - mean_absolute_error: 0.917664000/64000 [==============================] - 5s 72us/step - loss: 1.2552 - mean_absolute_error: 0.9184 - val_loss: 1.2968 - val_mean_absolute_error: 0.9474
Epoch 5/100
64000/64000 [==============================] - 5s 72us/step - loss: 1.2338 - mean_absolute_error: 0.9091 - val_loss: 1.2960 - val_mean_absolute_error: 0.9432
E

In [93]:
score =Gender_NCFG_model.evaluate([Users1,Movies1,Gender1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

16000/16000 [==============================] - 1s 35us/step
Test MSE: 1.5497420881986619
Test MAE: 1.001318374276161


### activation function = linear

In [94]:
K_LATENT = 20
hidden_dim = 100
do = 0.5
oprimaizer_name = 'nadam'
loss_function = 'mse'
activ_func ='linear'


Gender_NCFG_model = get_ncf_model_gender(max_userid,max_movieid,K_LATENT,hidden_dim,do)
Gender_NCFG_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()

callbacks_ncfgender = [EarlyStopping('val_loss', patience=20), 
             ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncfg_'+str(do)+'_'+str(K_LATENT)+'.h5', save_best_only=True)]
history = Gender_NCFG_model.fit([Users, Movies,Gender], Ratings, nb_epoch=100,validation_data=([Users1,Movies1,Gender1], Ratings1), validation_split=.2, verbose=1, callbacks=callbacks_ncfgender, batch_size = 32)


end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

Train on 64000 samples, validate on 16000 samples
Epoch 1/100
64000/64000 [==============================] - 6s 100us/step - loss: 1.6485 - mean_absolute_error: 1.0297 - val_loss: 1.2936 - val_mean_absolute_error: 0.9581
Epoch 2/100
64000/64000 [==============================] - 5s 75us/step - loss: 1.3758 - mean_absolute_error: 0.9568 - val_loss: 1.2719 - val_mean_absolute_error: 0.9478
Epoch 3/100
64000/64000 [==============================] - 5s 75us/step - loss: 1.3141 - mean_absolute_error: 0.9371 - val_loss: 1.2701 - val_mean_absolute_error: 0.9434
Epoch 4/100
15232/64000 [======>.......................] - ETA: 3s - loss: 1.2728 - mean_absolute_error: 0.922264000/64000 [==============================] - 5s 75us/step - loss: 1.2751 - mean_absolute_error: 0.9265 - val_loss: 1.2731 - val_mean_absolute_error: 0.9404
Epoch 5/100
64000/64000 [==============================] - 5s 75us/step - loss: 1.2494 - mean_absolute_error: 0.9216 - val_loss: 1.2821 - val_mean_absolute_error: 0.9474


In [95]:
score =Gender_NCFG_model.evaluate([Users1,Movies1,Gender1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

16000/16000 [==============================] - 1s 35us/step
Test MSE: 1.4337351348400116
Test MAE: 0.9757064599990845


### NCF and age as feature

In [96]:
###K_LATENT = 20
hidden_dim = 100
do = 0.1
oprimaizer_name = 'nadam'
loss_function = 'mae'
activ_func ='relu'


Age_NCFG_model = get_ncf_model_age(max_userid,max_movieid,K_LATENT,hidden_dim,do)
Age_NCFG_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()

callbacks_ncfAge = [EarlyStopping('val_loss', patience=20), 
             ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncfg_'+str(do)+'_'+str(K_LATENT)+'.h5', save_best_only=True)]
history = Age_NCFG_model.fit([Users, Movies,Age], Ratings, nb_epoch=100, validation_split=.2,validation_data=([Users1,Movies1,Age1], Ratings1), verbose=1, callbacks=callbacks_ncfAge, batch_size = 32)


end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

Train on 64000 samples, validate on 16000 samples
Epoch 1/100
64000/64000 [==============================] - 6s 101us/step - loss: 1.0096 - mean_absolute_error: 1.0096 - val_loss: 0.9127 - val_mean_absolute_error: 0.9127
Epoch 2/100
64000/64000 [==============================] - 5s 75us/step - loss: 0.9310 - mean_absolute_error: 0.9310 - val_loss: 0.9271 - val_mean_absolute_error: 0.9271
Epoch 3/100
64000/64000 [==============================] - 5s 75us/step - loss: 0.9198 - mean_absolute_error: 0.9198 - val_loss: 0.9197 - val_mean_absolute_error: 0.9197
Epoch 4/100
15136/64000 [======>.......................] - ETA: 3s - loss: 0.9067 - mean_absolute_error: 0.906764000/64000 [==============================] - 5s 76us/step - loss: 0.9100 - mean_absolute_error: 0.9100 - val_loss: 0.9319 - val_mean_absolute_error: 0.9319
Epoch 5/100
64000/64000 [==============================] - 5s 76us/step - loss: 0.8999 - mean_absolute_error: 0.8999 - val_loss: 0.9352 - val_mean_absolute_error: 0.9352


In [97]:
score =Age_NCFG_model.evaluate([Users1,Movies1,Age1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

16000/16000 [==============================] - 1s 36us/step
Test MSE: 0.9669711590409279
Test MAE: 0.9669711590409279


### drop-out = 0.5

In [98]:
k_LATENT = 20
hidden_dim = 100
do = 0.5
oprimaizer_name = 'nadam'
loss_function = 'mae'
activ_func ='relu'


Age_NCFG_model = get_ncf_model_age(max_userid,max_movieid,K_LATENT,hidden_dim,do)
Age_NCFG_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()

callbacks_ncfAge = [EarlyStopping('val_loss', patience=20), 
             ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncfg_'+str(do)+'_'+str(K_LATENT)+'.h5', save_best_only=True)]
history = Age_NCFG_model.fit([Users, Movies,Age], Ratings, nb_epoch=100, validation_split=.2,validation_data=([Users1,Movies1,Age1], Ratings1), verbose=1, callbacks=callbacks_ncfAge, batch_size = 32)


end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

Train on 64000 samples, validate on 16000 samples
Epoch 1/100
64000/64000 [==============================] - 7s 103us/step - loss: 1.0808 - mean_absolute_error: 1.0808 - val_loss: 0.9465 - val_mean_absolute_error: 0.9465
Epoch 2/100
64000/64000 [==============================] - 5s 75us/step - loss: 0.9520 - mean_absolute_error: 0.9520 - val_loss: 0.9391 - val_mean_absolute_error: 0.9391
Epoch 3/100
64000/64000 [==============================] - 5s 75us/step - loss: 0.9288 - mean_absolute_error: 0.9288 - val_loss: 0.9212 - val_mean_absolute_error: 0.9212
Epoch 4/100
14400/64000 [=====>........................] - ETA: 3s - loss: 0.9179 - mean_absolute_error: 0.917964000/64000 [==============================] - 5s 75us/step - loss: 0.9161 - mean_absolute_error: 0.9161 - val_loss: 0.9216 - val_mean_absolute_error: 0.9216
Epoch 5/100
64000/64000 [==============================] - 5s 75us/step - loss: 0.9042 - mean_absolute_error: 0.9042 - val_loss: 0.9210 - val_mean_absolute_error: 0.9210


In [99]:
score =Age_NCFG_model.evaluate([Users1,Movies1,Age1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

16000/16000 [==============================] - 1s 35us/step
Test MSE: 0.9247584543228149
Test MAE: 0.9247584543228149


### optimezer = adamax

In [100]:
K_LATENT = 20
hidden_dim = 100
do = 0.1
oprimaizer_name = 'adamax'
loss_function = 'mae'
activ_func ='relu'


Age_NCFG_model = get_ncf_model_age(max_userid,max_movieid,K_LATENT,hidden_dim,do)
Age_NCFG_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()

callbacks_ncfAge = [EarlyStopping('val_loss', patience=20), 
             ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncfg_'+str(do)+'_'+str(K_LATENT)+'.h5', save_best_only=True)]
history = Age_NCFG_model.fit([Users, Movies,Age], Ratings, nb_epoch=100, validation_split=.2,validation_data=([Users1,Movies1,Age1], Ratings1), verbose=1, callbacks=callbacks_ncfAge, batch_size = 32)


end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

Train on 64000 samples, validate on 16000 samples
Epoch 1/100
64000/64000 [==============================] - 6s 96us/step - loss: 1.0944 - mean_absolute_error: 1.0944 - val_loss: 0.9317 - val_mean_absolute_error: 0.9317
Epoch 2/100
64000/64000 [==============================] - 4s 68us/step - loss: 0.9278 - mean_absolute_error: 0.9278 - val_loss: 0.9138 - val_mean_absolute_error: 0.9138
Epoch 3/100
64000/64000 [==============================] - 4s 67us/step - loss: 0.9188 - mean_absolute_error: 0.9188 - val_loss: 0.9158 - val_mean_absolute_error: 0.9158
Epoch 4/100
28896/64000 [============>.................] - ETA: 2s - loss: 0.9106 - mean_absolute_error: 0.910664000/64000 [==============================] - 4s 68us/step - loss: 0.9155 - mean_absolute_error: 0.9155 - val_loss: 0.9227 - val_mean_absolute_error: 0.9227
Epoch 5/100
64000/64000 [==============================] - 4s 67us/step - loss: 0.9130 - mean_absolute_error: 0.9130 - val_loss: 0.9359 - val_mean_absolute_error: 0.9359
E

In [101]:
score =Age_NCFG_model.evaluate([Users1,Movies1,Age1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

16000/16000 [==============================] - 1s 35us/step
Test MSE: 0.9384622576236725
Test MAE: 0.9384622576236725


### loss function = mse

In [102]:
K_LATENT = 20
hidden_dim = 100
do = 0.1
oprimaizer_name = 'nadam'
loss_function = 'mse'
activ_func ='relu'


Age_NCFG_model = get_ncf_model_age(max_userid,max_movieid,K_LATENT,hidden_dim,do)
Age_NCFG_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()

callbacks_ncfAge = [EarlyStopping('val_loss', patience=20), 
             ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncfg_'+str(do)+'_'+str(K_LATENT)+'.h5', save_best_only=True)]
history = Age_NCFG_model.fit([Users, Movies,Age], Ratings, nb_epoch=100,validation_data=([Users1,Movies1,Age1], Ratings1), validation_split=.2, verbose=1, callbacks=callbacks_ncfAge, batch_size = 32)


end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

Train on 64000 samples, validate on 16000 samples
Epoch 1/100
64000/64000 [==============================] - 7s 105us/step - loss: 1.5982 - mean_absolute_error: 1.0216 - val_loss: 1.2663 - val_mean_absolute_error: 0.9489
Epoch 2/100
64000/64000 [==============================] - 5s 76us/step - loss: 1.2940 - mean_absolute_error: 0.9376 - val_loss: 1.3182 - val_mean_absolute_error: 0.9604
Epoch 3/100
64000/64000 [==============================] - 5s 76us/step - loss: 1.2723 - mean_absolute_error: 0.9281 - val_loss: 1.2746 - val_mean_absolute_error: 0.9398
Epoch 4/100
14336/64000 [=====>........................] - ETA: 3s - loss: 1.2331 - mean_absolute_error: 0.911464000/64000 [==============================] - 5s 76us/step - loss: 1.2543 - mean_absolute_error: 0.9197 - val_loss: 1.2985 - val_mean_absolute_error: 0.9350
Epoch 5/100
64000/64000 [==============================] - 5s 76us/step - loss: 1.2382 - mean_absolute_error: 0.9132 - val_loss: 1.3301 - val_mean_absolute_error: 0.9634


In [103]:
score =Age_NCFG_model.evaluate([Users1,Movies1,Age1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

16000/16000 [==============================] - 1s 36us/step
Test MSE: 1.4981908673048019
Test MAE: 0.9899061049222946


### activation function = selu layer size 20

In [104]:
###K_LATENT = 20
hidden_dim = 20
do = 0.1
oprimaizer_name = 'nadam'
loss_function = 'mae'
activ_func ='selu'


Age_NCFG_model = get_ncf_model_age(max_userid,max_movieid,K_LATENT,hidden_dim,do)
Age_NCFG_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()

callbacks_ncfAge = [EarlyStopping('val_loss', patience=20), 
             ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncfg_'+str(do)+'_'+str(K_LATENT)+'.h5', save_best_only=True)]
history = Age_NCFG_model.fit([Users, Movies,Age], Ratings, nb_epoch=100,validation_data=([Users1,Movies1,Age1], Ratings1), validation_split=.2, verbose=1, callbacks=callbacks_ncfAge, batch_size = 32)


end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

Train on 64000 samples, validate on 16000 samples
Epoch 1/100
64000/64000 [==============================] - 7s 103us/step - loss: 1.0893 - mean_absolute_error: 1.0893 - val_loss: 0.9270 - val_mean_absolute_error: 0.9270
Epoch 2/100
64000/64000 [==============================] - 5s 74us/step - loss: 0.9290 - mean_absolute_error: 0.9290 - val_loss: 0.9208 - val_mean_absolute_error: 0.9208
Epoch 3/100
64000/64000 [==============================] - 5s 73us/step - loss: 0.9225 - mean_absolute_error: 0.9225 - val_loss: 0.9230 - val_mean_absolute_error: 0.9230
Epoch 4/100
17600/64000 [=======>......................] - ETA: 3s - loss: 0.9181 - mean_absolute_error: 0.918164000/64000 [==============================] - 5s 74us/step - loss: 0.9134 - mean_absolute_error: 0.9134 - val_loss: 0.9377 - val_mean_absolute_error: 0.9377
Epoch 5/100
64000/64000 [==============================] - 5s 74us/step - loss: 0.9054 - mean_absolute_error: 0.9054 - val_loss: 0.9123 - val_mean_absolute_error: 0.9123


In [105]:
score =Age_NCFG_model.evaluate([Users1,Movies1,Age1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

16000/16000 [==============================] - 1s 36us/step
Test MSE: 0.9298802262544632
Test MAE: 0.9298802262544632


### activation function = linear

In [106]:
###K_LATENT = 20
hidden_dim = 100
do = 0.1
oprimaizer_name = 'nadam'
loss_function = 'mae'
activ_func ='linear'


Age_NCFG_model = get_ncf_model_age(max_userid,max_movieid,K_LATENT,hidden_dim,do)
Age_NCFG_model.compile(loss=loss_function,optimizer=oprimaizer_name,metrics=['mae'])
ini = time.time()

callbacks_ncfAge = [EarlyStopping('val_loss', patience=20), 
             ModelCheckpoint(MODEL_WEIGHTS_FILE_CORE+'_ncfg_'+str(do)+'_'+str(K_LATENT)+'.h5', save_best_only=True)]
history = Age_NCFG_model.fit([Users, Movies,Age], Ratings, nb_epoch=100,validation_data=([Users1,Movies1,Age1], Ratings1), validation_split=.2, verbose=1, callbacks=callbacks_ncfAge, batch_size = 32)


end = time.time()
print ("Total time : ",end-ini)
print("MAE : " , min(history.history['val_loss']))                                    

Train on 64000 samples, validate on 16000 samples
Epoch 1/100
64000/64000 [==============================] - 7s 107us/step - loss: 1.0132 - mean_absolute_error: 1.0132 - val_loss: 0.9418 - val_mean_absolute_error: 0.9418
Epoch 2/100
64000/64000 [==============================] - 5s 76us/step - loss: 0.9257 - mean_absolute_error: 0.9257 - val_loss: 0.9179 - val_mean_absolute_error: 0.9179
Epoch 3/100
64000/64000 [==============================] - 5s 76us/step - loss: 0.9166 - mean_absolute_error: 0.9166 - val_loss: 0.9223 - val_mean_absolute_error: 0.9223
Epoch 4/100
13312/64000 [=====>........................] - ETA: 3s - loss: 0.9006 - mean_absolute_error: 0.900664000/64000 [==============================] - 5s 77us/step - loss: 0.9077 - mean_absolute_error: 0.9077 - val_loss: 0.9248 - val_mean_absolute_error: 0.9248
Epoch 5/100
64000/64000 [==============================] - 5s 76us/step - loss: 0.8989 - mean_absolute_error: 0.8989 - val_loss: 0.9368 - val_mean_absolute_error: 0.9368


In [107]:
score =Age_NCFG_model.evaluate([Users1,Movies1,Age1], Ratings1,verbose=1)
print('Test MSE:', score[0])
print('Test MAE:', score[1])

16000/16000 [==============================] - 1s 37us/step
Test MSE: 0.9872714752554893
Test MAE: 0.9872714752554893
